In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
print(tf.__version__)
#tf.config.list_physical_devices('GPU')
tf.test.gpu_device_name()

2.4.0


'/device:GPU:0'

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from string import punctuation
#from autocorrect import spell
import string
snowball_stemmer = SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
!pip install autocorrect

     |████████████████████████████████| 624kB 7.1MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.2.2-cp36-none-any.whl size=621492 sha256=81d83eff706788a573844005702a025a7400c427633e389c7e2a0e37830084a1
  Stored in directory: /root/.cache/pip/wheels/b4/0b/7d/98268d64c8697425f712c897265394486542141bbe4de319d6
Successfully built autocorrect


In [ ]:
import pandas as pd
import numpy as np

data=pd.read_csv('/content/gdrive/My Drive/CVAssignmentDataset.csv')
df=pd.read_csv('/content/gdrive/My Drive/CVAssignmentDataset.csv')
df = df.reindex(np.random.permutation(df.index))
df = df[['Class', 'Text']]
data4=pd.read_csv('/content/gdrive/My Drive/data_prepr.csv')

In [ ]:
df.head()
df.dropna(inplace=True)
print(len(df))

86460


In [ ]:
text=df["Text"]
label=df["Class"]

In [ ]:
from functools import wraps
from unicodedata import normalize
from typing import List, Optional, Union, Callable
from pathlib import Path
import csv
# Third party libraries

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, WordNetLemmatizer

from names_dataset import NameDataset
import re
import string

In [ ]:
!pip install names_dataset

     |████████████████████████████████| 1.4MB 8.8MB/s 


In [ ]:
sb_stem = SnowballStemmer("english", ignore_stopwords=True)
pt_stem = PorterStemmer()

lmtzr = WordNetLemmatizer()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
_CUSTOM_SUB_CSV_FILE_PATH = ('/content/gdrive/My Drive/custom_substitutions.csv')
_IGNORE_SPELLCHECK_WORD_FILE_PATH = ('/content/gdrive/My Drive/ignore_spellcheck_words.txt')
def _return_empty_string_for_invalid_input(func):
    """ Return empty string if the input is None or empty """
    @wraps(func)
    def wrapper(*args, **kwargs):
        if 'input_text' in kwargs:
            input_text = kwargs['input_text']
        else:
            try:
                input_text = args[0]
            except IndexError as e:
                LOGGER.exception('No appropriate positional argument is provide.')
                raise e
        if input_text is None or len(input_text) == 0:
            return ''
        else:
            return func(*args, **kwargs)
    return wrapper



def _return_empty_list_for_invalid_input(func):
    """ Return empty list if the input is None or empty """
    @wraps(func)
    def wrapper(*args, **kwargs):
        if 'input_text_or_list' in kwargs:
            input_text_or_list = kwargs['input_text_or_list']
        else:
            try:
                input_text_or_list = args[0]
            except IndexError as e:
                LOGGER.exception('No appropriate positional argument is provide.')
                raise e
        if input_text_or_list is None or len(input_text_or_list) == 0:
            return []
        else:
            return func(*args, **kwargs)
    return wrapper


@_return_empty_string_for_invalid_input
def to_lower(input_text: str) -> str:
    """ Convert input text to lower case """
    return input_text.lower()


@_return_empty_string_for_invalid_input
def to_upper(input_text: str) -> str:
    """ Convert input text to upper case """
    return input_text.upper()


@_return_empty_string_for_invalid_input
def remove_number(input_text: str) -> str:
    """ Remove number in the input text """
    processed_text = re.sub('\d+', '', input_text)
    return processed_text


@_return_empty_string_for_invalid_input
def remove_itemized_bullet_and_numbering(input_text: str) -> str:
    """ Remove bullets or numbering in itemized input """
    processed_text = re.sub('[(\s][0-9a-zA-Z][.)]\s+|[(\s][ivxIVX]+[.)]\s+', ' ', input_text)
    return processed_text


@_return_empty_string_for_invalid_input
def remove_url(input_text: str) -> str:
    """ Remove url in the input text """
    return re.sub('(www|http)\S+', '', input_text)


@_return_empty_string_for_invalid_input
def remove_punctuation(input_text: str, punctuations: Optional[str] = None) -> str:
    """
    Removes all punctuations from a string, as defined by string.punctuation or a custom list.
    For reference, Python's string.punctuation is equivalent to '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'
    """
    if punctuations is None:
        punctuations = string.punctuation
    processed_text = input_text.translate(str.maketrans('', '', punctuations))
    return processed_text


@_return_empty_string_for_invalid_input
def remove_special_character(input_text: str, special_characters: Optional[str] = None) -> str:
    """ Removes special characters """
    if special_characters is None:
        # TODO: add more special characters
        special_characters = 'å¼«¥ª°©ð±§µæ¹¢³¿®ä£'
    processed_text = input_text.translate(str.maketrans('', '', special_characters))
    return processed_text


@_return_empty_string_for_invalid_input
def keep_alpha_numeric(input_text: str) -> str:
    """ Remove any character except alphanumeric characters """
    return ''.join(c for c in input_text if c.isalnum())


@_return_empty_string_for_invalid_input
def remove_whitespace(input_text: str, remove_duplicate_whitespace: bool = True) -> str:
    """ Removes leading, trailing, and (optionally) duplicated whitespace """
    if remove_duplicate_whitespace:
        return ' '.join(re.split('\s+', input_text.strip(), flags=re.UNICODE))
    return input_text.strip()

def substitute_token(token_list: List[str], sub_dict: Optional[dict] = None) -> List[str]:
    """ Substitute each token by another token, e.g., 'vs' -> 'versus' """
    # TODO: add more custom substitutions in the csv file specified by _CUSTOM_SUB_CSV_FILE_PATH
    if token_list is None or len(token_list) == 0:
        return []
    if sub_dict is None:
        with open(_CUSTOM_SUB_CSV_FILE_PATH, 'r') as f:
            csv_file = csv.reader(f)
            sub_dict = dict(csv_file)
    processed_tokens = list()
    for token in token_list:
        if token in sub_dict:
            processed_tokens.append(sub_dict[token])
        else:
            processed_tokens.append(token)
    return processed_tokens

@_return_empty_string_for_invalid_input
def normalize_unicode(input_text: str) -> str:
    """ Normalize unicode data to remove umlauts, and accents, etc. """
    processed_tokens = normalize('NFKD', input_text).encode('ASCII', 'ignore').decode('utf8')
    return processed_tokens


@_return_empty_list_for_invalid_input
def remove_stopword(input_text_or_list: Union[str, List[str]], stop_words: Optional[set] = None) -> List[str]:
    """ Remove stop words """

    if stop_words is None:
        stop_words = set(stopwords.words('english'))
    if isinstance(stop_words, list):
        stop_words = set(stop_words)
    if isinstance(input_text_or_list, str):
        tokens = word_tokenize(input_text_or_list)
        processed_tokens = [token for token in tokens if token not in stop_words]
    else:
        processed_tokens = [token for token in input_text_or_list
                            if (token not in stop_words and token is not None and len(token) > 0)]
    return processed_tokens


@_return_empty_string_for_invalid_input
def remove_email(input_text: str) -> str:
    """ Remove email in the input text """
    regex_pattern = '[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}'
    return re.sub(regex_pattern, '', input_text)


@_return_empty_string_for_invalid_input
def remove_phone_number(input_text: str) -> str:
    """ Remove phone number in the input text """
    regex_pattern = '(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})(?: *x(\d+))?'
    return re.sub(regex_pattern, '', input_text)


@_return_empty_string_for_invalid_input
def remove_ssn(input_text: str) -> str:
    """ Remove social security number in the input text """
    regex_pattern = '(?!219-09-9999|078-05-1120)(?!666|000|9\d{2})\d{3}-(?!00)\d{2}-(?!0{4})\d{4}|(' \
                    '?!219099999|078051120)(?!666|000|9\d{2})\d{3}(?!00)\d{2}(?!0{4})\d{4}'
    return re.sub(regex_pattern, '', input_text)


@_return_empty_string_for_invalid_input
def remove_credit_card_number(input_text: str) -> str:
    """ Remove credit card number in the input text """
    regex_pattern = '(4[0-9]{12}(?:[0-9]{3})?|(?:5[1-5][0-9]{2}|222[1-9]|22[3-9][0-9]|2[3-6][0-9]{2}|27[01][' \
                    '0-9]|2720)[0-9]{12}|3[47][0-9]{13}|3(?:0[0-5]|[68][0-9])[0-9]{11}|6(?:011|5[0-9]{2})[0-9]{12}|(' \
                    '?:2131|1800|35\d{3})\d{11})'
    return re.sub(regex_pattern, '', input_text)


@_return_empty_list_for_invalid_input
def remove_name(input_text_or_list: Union[str, List[str]]) -> List[str]:
    """ Remove name in the input text """
    name_searcher = NameDataset()
    if isinstance(input_text_or_list, str):
        tokens = word_tokenize(input_text_or_list)
        processed_tokens = [token for token in tokens
                            if (not name_searcher.search_first_name(token)) and
                               (not name_searcher.search_last_name(token))]
    else:
        processed_tokens = [token for token in input_text_or_list
                            if (not name_searcher.search_first_name(token)) and
                               (not name_searcher.search_last_name(token)) and token is not None and len(token) > 0]
    return processed_tokens




def tokenize_word(input_text: str) -> List[str]:
    """ Converts a text into a list of word tokens """
    if input_text is None or len(input_text) == 0:
        return []
    return word_tokenize(input_text)


def tokenize_sentence(input_text: str) -> List[str]:
    """ Converts a text into a list of sentence tokens """
    if input_text is None or len(input_text) == 0:
        return []
    tokenizer = PunktSentenceTokenizer()
    return tokenizer.tokenize(input_text)


@_return_empty_list_for_invalid_input
def stem_word(input_text_or_list: Union[str, List[str]],
              stemmer: Optional[Union[PorterStemmer, SnowballStemmer, LancasterStemmer]] = None
              ) -> List[str]:
    """ Stem each token in a text """
    if stemmer is None:
        stemmer = PorterStemmer()
    if isinstance(input_text_or_list, str):
        tokens = word_tokenize(input_text_or_list)
        processed_tokens = [stemmer.stem(token) for token in tokens]
    else:
        processed_tokens = [stemmer.stem(token) for token in input_text_or_list if token is not None and len(token) > 0]
    return processed_tokens


@_return_empty_list_for_invalid_input
def lemmatize_word(input_text_or_list: Union[str, List[str]],
                   lemmatizer: Optional[WordNetLemmatizer] = None
                   ) -> List[str]:
    """ Lemmatize each token in a text by finding its base form """
    if lemmatizer is None:
        lemmatizer = WordNetLemmatizer()
    if isinstance(input_text_or_list, str):
        tokens = word_tokenize(input_text_or_list)
        processed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    else:
        processed_tokens = [lemmatizer.lemmatize(token)
                            for token in input_text_or_list if token is not None and len(token) > 0]
    return processed_tokens
def remove_words_start_with(text, starts_with_char):
    """
    Remove words start with character `starts_with_char`
    Example: dhoni rocks with last ball six #dhoni #six => dhoni rocks with last ball six (start_char_with='#')
    Args:
        text (str): text
        starts_with_char (str): starting characters of word, which to be removed from text
    Returns:
        text (str): text with removed words start with given chars
    """
    urls = re.finditer(starts_with_char + r'[A-Za-z0-9\w]*', text)
    for i in urls:
        text = re.sub(i.group().strip(), '', text)
    return text.strip()
def remove_alphanumerics(text):
    """
    Remove alphanumeric words from text
    Example: hello man whatsup123 => hello man
    Args:
        text (str): text
    Returns:
        text (str): text with removed alphanumeric words
    """
    txt = []
    for each in text.split():
        if not any(x in each.lower() for x in "0123456789"):
            txt.append(each)
    txtsent = " ".join(txt)
    return txtsent 
def remove_extra_space(text):
    """
    Remove extra white spaces space from text
    Example: hey are   you coming. ? => he are you coming. ?
    Args:
        text (str): text
    Returns:
        clean_text (str): clean text with removed extra white spaces
    """
    clean_text = ' '.join(text.strip().split())
    return clean_text
def slang_look_up(text):
    """
    Replace slang word in text to their original form
    Example: hi, thanq so mch => hi, thank you so much
    Args:
        text (str): text
    Returns:
        slanged (str): cleaned text with replaced slang
    """
    words = text.split()
    new_text = []

    for word in words:
        word_s = word.lower()
        if word_s in slangs_dict:
            new_text.append(slangs_dict[word_s])
        else:
            new_text.append(word)
    slanged = " ".join(new_text)
    return slanged
def separate_digit_text(text):
    """
    Separate digit and words with space in text
    Example: I will be booking tickets for 2adults => I will be booking tickets for 2 adults   
    Args:
        text (str): text
    Returns:
        clean_text (str): cleaned text with separated digits and words
    """
    regex_patter = re.compile(r'([\d]+)([a-zA-Z]+)')
    clean_text = regex_patter.sub(r'\1 \2', text)
    return clean_text
def remove_repeated_characters(text):
    """
    Remove repeated characters (>2) in words to max limit of 2
    Example: I am verrry happpyyy today => I am verry happyy today
    Args:
        text (str): text
    Returns:
        clean_text (str): cleaned text with removed repeated chars
    """
    regex_pattern = re.compile(r'(.)\1+')
    clean_text = regex_pattern.sub(r'\1\1', text)
    return clean_text


def appos_look_up(text):
    """
    Convert apostrophes word to original form
    Example: I don't know what is going on?  => I do not know what is going on? 
    Args:
        text (str): text 
    Returns:
        apposed (str) : text with converted apostrophes
    """
    words = text.split()
    new_text = []
    for word in words:
        word_s = word.lower()
        if word_s in appos_dict:
            new_text.append(appos_dict[word_s])
        else:
            new_text.append(word)
    apposed = " ".join(new_text)
    return apposed
def remove_single_char_word(text):
    """
    Remove single character word from text
    Example: I am in a home for 2 years => am in home for years 
    Args:
        text (str): text
         
    Returns:
        (str): text with single char removed
    """
    words = text.split()
    filter_words = [word for word in words if len(word) > 1]
    return " ".join(filter_words)
def remove_words_start_with(text):
    """
    Remove words start with character `starts_with_char`
    Example: dhoni rocks with last ball six #dhoni #six => dhoni rocks with last ball six (start_char_with='#')
    Args:
        text (str): text
        starts_with_char (str): starting characters of word, which to be removed from text
    Returns:
        text (str): text with removed words start with given chars
    """
    urls = re.finditer('#' + r'[A-Za-z0-9\w]*', text)
    for i in urls:
        text = re.sub(i.group().strip(), '', text)
    return text.strip()
def remove_words_start(text):
    """
    Remove words start with character `starts_with_char`
    Example: dhoni rocks with last ball six #dhoni #six => dhoni rocks with last ball six (start_char_with='#')
    Args:
        text (str): text
        starts_with_char (str): starting characters of word, which to be removed from text
    Returns:
        text (str): text with removed words start with given chars
    """
    urls = re.finditer('@' + r'[A-Za-z0-9\w]*', text)
    for i in urls:
        text = re.sub(i.group().strip(), '', text)
    return text.strip()
def emoticons_look_up(text):
    """
    Remove emoticons from text and returns list of emotions present in text
    Example: Sure, you are welcome :) => Sure, you are welcome.
    Args:
        text (str): text
    Returns:
        text (str): text with removed emoticons sign
        emolist (list) : list of emotions from text
    """

    words = text.split()
    newlist = []
    for word in words:
          if word in emo:            
            text1=emoticons[word]
            newlist.append(text1)
          else:
            newlist.append(word)
    text = " ".join(newlist)
    return text

def preprocess_text(input_text: str, processing_function_list: Optional[List[Callable]] = None) -> str:
    """ Preprocess an input text by executing a series of preprocessing functions specified in functions list """
    if processing_function_list is None:
        processing_function_list = [to_lower,
                                    remove_url,
                                    remove_email,
                                    remove_phone_number,
                                    remove_credit_card_number,
                                    remove_ssn,
                                    remove_itemized_bullet_and_numbering,                                 
                                    emoticons_look_up,
                                    #appos_look_up,
                                    #slang_look_up,                                    
                                    remove_alphanumerics,                                    
                                    remove_extra_space,
                                    remove_repeated_characters,
                                    remove_words_start_with,
                                    remove_words_start,
                                    separate_digit_text,                                    
                                    remove_special_character,
                                    remove_punctuation,
                                    remove_whitespace,
                                    normalize_unicode,
                                    remove_single_char_word,
                                    #remove_stopword,
                                    remove_name,                                    
                                    #substitute_token,                                    
                                    #lemmatize_word,
                                    #stem_word
                                    ]
    for func in processing_function_list:
        input_text = func(input_text)
    if isinstance(input_text, str):
        processed_text = input_text
    else:
        processed_text = ' '.join(input_text)
    return processed_text


In [ ]:
#df.dropna()
#df.dropna(inplace=True)





In [ ]:
from deep_translator import GoogleTranslator
list1=[]
for i in text:
 
 preprocessed_text = preprocess_text(i)
 #data_tr["Text"][i]=preprocessed_text
 list1.append(preprocessed_text)
 

In [ ]:
#print((data_tr["Text"]))
print(len(list1))
d = pd.DataFrame(list1)
print(len(label))
label.to_csv("/content/gdrive/My Drive/label.csv")
d.to_csv("/content/gdrive/My Drive/data_p.csv")
#d.to_csv("/content/gdrive/My Drive/data_p.csv")



86460
86460


In [ ]:

import pandas as pd
data4=pd.read_csv('/content/gdrive/My Drive/data_prepr.csv')
data4['word_count'] = data4['Text'].apply(lambda x: len(str(x).split(" ")))
print (len(data4[data4["word_count"] >=3]))
data_c=data4[data4["word_count"] >=3]
print(data_c)

84917
                                                    Text  Class  word_count
0      solutions like tax reform and trade promotion ...      0          18
1      sc has seen years wconsecutive federal disaste...      0          14
2         rt fantastic mt united in introduction of peak      0           8
3      thank you for honoring me among this years lis...      1          19
4      let us take the time to remember those heroic ...      1          24
...                                                  ...    ...         ...
86455  rt has denied the science of and let environme...      1          13
86456  anything you have to say about the amendment a...      1          22
86457  rt rm today we welcome back to america three w...      1          15
86458  its on all of us to create culture where viole...      1          20
86459   rt the award goes to his leadership to inspi ...      1          10

[84917 rows x 3 columns]


In [ ]:
text1=data_c["Text"]
label1=data_c["Class"]
print(len(text1))

84917


In [ ]:
def preprocess_text(input_text: str, processing_function_list: Optional[List[Callable]] = None) -> str:
    """ Preprocess an input text by executing a series of preprocessing functions specified in functions list """
    if processing_function_list is None:
        processing_function_list = [                                    
                                    appos_look_up,
                                    slang_look_up,                                 
                                    
                                    remove_single_char_word,
                                    remove_stopword,
                                    remove_name,                                    
                                    substitute_token,                                    
                                    lemmatize_word,
                                    stem_word
                                    ]
    for func in processing_function_list:
        input_text = func(input_text)
    if isinstance(input_text, str):
        processed_text = input_text
    else:
        processed_text = ' '.join(input_text)
    return processed_text

In [ ]:
from deep_translator import GoogleTranslator

list1=[]
for i in text1:
 
 translated1 = GoogleTranslator(source='auto', target='en').translate(i)
 
 preprocessed_text = preprocess_text(translated1)
 list1.append(preprocessed_text)
 
 

In [ ]:
data_c_pre=pd.DataFrame(list1) 
data_c_pre.to_csv("/content/gdrive/My Drive/complete_pre.csv")
label1.to_csv("/content/gdrive/My Drive/Class_pre.csv")

In [ ]:
!pip install deep_translator
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
slangs_dict = {
    'awsm': 'awesome',
    "aamof": "as a matter of fact",
    "abt": "about",
    "abt2": "about to",
    "ac": "air conditioning",
    "ace": "solo winner",
    "ack": "acknowledged",
    "admin": "administrator",
    "thr": "there",
    "frm": "from",
    "aggro": "aggression",
    "agl": "angel",
    "dob": "date of birth",
    "ai": "artificial intelligence",
    "aiic": "as if i care",
    "aka": "also known as",
    "alap": "as long as possible",
    "alol": "actually laughing out loud",
    "ama": "ask me anything",
    "amap": "as much as possible",
    "amazn": "amazing",
    "ammo": "ammunition",
    "ams": "ask me something",
    "anon": "anonymous",
    "asap": "as soon as possible",
    "asat": "as simple as that",
    "awks": "awkward",
    "awl": "always with love",
    "ayk": "as you know",
    "azm": "awesome",
    "b": "be",
    "b&w": "black and white",
    "b-day": "birthday",
    "bday": "birthday",
    "bcoz": "because",
    "bcos": "because",
    "bcz": "because",
    "bf": "boyfriend",
    "btw": "between",
    "b4": "before",
    "bai": "bye",
    "bb": "bye bye",
    "bc": "abuse",
    "mc": "abuse",
    "bcc": "blind carbon copy",
    "bff": "best friends forever",
    "biz": "business",
    "bk": "back",
    "bo": "back off",
    "bro": "brother",
    "btwn": "between",
    "c": "see",
    "char": "character",
    "combo": "combination",
    "cu": "see you",
    "cu2": "see you too",
    "cu2mr": "see you tomorrow",
    "cya": "see ya",
    "cyal8r": "see you later",
    "cyb": "call you back",
    "cye": "check your e-mail",
    "cyf": "check your facebook",
    "cyfb": "check your facebook",
    "cyl": "catch ya later",
    "cym": "check your myspace",
    "cyo": "see you online",
    "d8": "date",
    "da": "the",
    "dece": "decent",
    "ded": "dead",
    "dept": "department",
    "dis": "this",
    "ditto": "same",
    "diva": "rude woman",
    "dk": "don't know",
    "dlm": "don't leave me",
    "dm": "direct message",
    "dnd": "do not disturb",
    "dno": "don't know",
    "dnt": "don't",
    "e1": "everyone",
    "eg": "for example",
    "emc2": "genius",
    "emo": "emotional",
    "enuf": "enough",
    "eod": "end of discussion",
    "eof": "end of file",
    "eom": "end of message",
    "eta": "estimated time of arrival",
    "every1": "everyone",
    "evs": "whatever",
    "exp": "experience",
    "f": "female",
    "f2f": "face to face",
    "f2p": "free to play",
    "f2t": "free to talk",
    "f9": "fine",
    "fab": "fabulous",
    "fail": "failure",
    "faq": "frequently asked questions",
    "fav": "favorite",
    "fave": "favorite",
    "favs": "favorites",
    "fb": "facebook",
    "fbc": "facebook chat",
    "fbf": "facebook friend",
    "fml": "family",
    "fn": "fine",
    "fo": "freaking out",
    "fri": "friday",
    "frnd": "friend",
    "fu": "fuck you",
    "fugly": "fucking ugly",
    "gf": "girlfriend",
    "g1": "good one",
    "g2b": "going to bed",
    "g2cu": "good to see you",
    "g2g": "good to go",
    "g4i": "go for it",
    "g4n": "good for nothing",
    "g4u": "good for you",
    "g9": "goodnight",
    "ga": "go ahead",
    "ge": "good evening",
    "gl": "good luck",
    "gm": "good morning",
    "gn": "goodnight",
    "gonna": "going to",
    "goon": "idiot",
    "gorge": "gorgeous",
    "gr8": "great",
    "grats": "congratulations",
    "gratz": "congratulations",
    "grl": "girl",
    "gt2t": "got time to talk",
    "gtg": "good to go",
    "gud": "good",
    "gv": "give",
    "gvn": "given",
    "gw": "good work",
    "h/o": "hold on",
    "h/p": "hold please",
    "h/t": "hat tip",
    "h/u": "hook up",
    "h2cus": "hope to see you soon",
    "h4u": "hot for you",
    "h4x0r": "hacker",
    "h4x0rz": "hackers",
    "h8": "hate",
    "h8r": "hater",
    "h8t": "hate",
    "ha": "hello again",
    "haha": "laughing",
    "hai": "hi",
    "hak": "hugs and kisses",
    "han": "how about now?",
    "hav": "have",
    "hax": "hacks",
    "haxor": "hacker",
    "hay": "how are you",
    "hb2u": "happy birthday to you",
    "hbbd": "happy belated birthday",
    "hbd": "happy birthday",
    "hc": "how cool",
    "hcit": "how cool is that",
    "hehe": "laughing",
    "hf": "have fun",
    "hi5": "high five",
    "hig": "how's it going?",
    "hih": "hope it helps",
    "ho": "hold on",
    "hoc": "house of cards",
    "hof": "hall of fame",
    "holla": "holler",
    "hom": "hit or miss",
    "hood": "neighborhood",
    "hoops": "basketball",
    "hottie": "attractive person",
    "hr": "human resources",
    "hru": "how are you",
    "hry": "hurry",
    "hubby": "husband",
    "hwk": "homework",
    "hwp": "height weight proportionate",
    "hwu": "hey, what's up?",
    "hxc": "hardcore",
    "h^": "hook up",
    "i8": "i ate",
    "i8u": "i hate you",
    "ia": "i agree",
    "iab": "in a bit",
    "iac": "in any case",
    "iad": "it all depends",
    "iae": "in any event",
    "iag": "it's all good",
    "iagw": "in a good way",
    "iail": "i am in love",
    "iam": "in a minute",
    "ic": "i see",
    "id10t": "idiot",
    "idc": "i don't care",
    "idd": "indeed",
    "idi": "i doubt it",
    "idk": "i don't know",
    "idky": "i don't know why",
    "idmb": "i'll do my best",
    "idn": "i don't know",
    "idnk": "i do not know",
    "idr": "i don't remember",
    "idt": "i don't think",
    "idts": "i don't think so",
    "idtt": "i'll drink to that",
    "idu": "i don't understand",
    "ie": "that is",
    "ig2p": "i got to pee",
    "iggy": "ignored",
    "ight": "alright",
    "igi": "i get it",
    "ign": "in-game name",
    "igtp": "i get the point",
    "ih8u": "i hate you",
    "ihu": "i hate you",
    "ihy": "i hate you",
    "ii": "i'm impressed",
    "iiok": "if i only knew",
    "iir": "if i remember",
    "iirc": "if i remember correctly",
    "iit": "i'm impressed too",
    "iiuc": "if i understand correctly",
    "ik": "i know",
    "ikhyf": "i know how you feel",
    "ikr": "i know, right?",
    "ikwum": "i know what you mean",
    "ikwym": "i know what you mean",
    "ikyd": "i know you did",
    "ilu": "i like you",
    "ilu2": "i love you too",
    "ilub": "i love you baby",
    "ilyk": "i'll let you know",
    "ilyl": "i love you lots",
    "ilysm": "i love you so much",
    "ima": "i'm",
    "imma": "i'm gonna",
    "imo": "in my opinion",
    "imy": "i miss you",
    "inb4": "in before",
    "inc": "incoming",
    "indie": "independent",
    "info": "information",
    "init": "initialize",
    "ipo": "initial public offering",
    "ir": "in room",
    "ir8": "irate",
    "irdk": "i really don't know",
    "irl": "in real life",
    "iyo": "in your opinion",
    "iyq": "i like you",
    "j/k": "just kidding",
    "j/p": "just playing",
    "j/w": "just wondering",
    "j2lyk": "just to let you know",
    "j4f": "just for fun",
    "j4g": "just for grins",
    "jas": "just a second",
    "jb/c": "just because",
    "joshing": "joking",
    "k": "ok",
    "k3u": "i love you",
    "kappa": "sarcasm",
    "kek": "korean laugh",
    "keke": "korean laugh",
    "kewl": "cool",
    "kewt": "cute",
    "kfc": "kentucky fried chicken",
    "kgo": "ok, go",
    "kik": "laughing out loud",
    "kinda": "kind of",
    "kk": "ok",
    "kl": "kool",
    "km": "kiss me",
    "kma": "kiss my ass",
    "knp": "ok, no problem",
    "kw": "know",
    "kwl": "cool",
    "l2m": "listening to music",
    "l2p": "learn to play",
    "l33t": "leet",
    "l8": "late",
    "l8er": "later",
    "l8r": "later",
    "la": "laughing a lot",
    "laf": "laugh",
    "laffing": "laughing",
    "lafs": "love at first sight",
    "lam": "leave a message",
    "lamer": "lame person",
    "legit": "legitimate",
    "lemeno": "let me know",
    "lil": "little",
    "lk": "like",
    "llol": "literally laughing out loud",
    "lmho": "laughing my head off",
    "loi": "laughing on the inside",
    "lola": "love often, laugh a lot",
    "lolol": "lots of laugh out louds",
    "lolz": "laugh out louds",
    "ltr": "later",
    "lulz": "lol",
    "luv": "love",
    "luzr": "loser",
    "lv": "love",
    "ly": "love ya",
    "lya": "love you always",
    "lyk": "let you know",
    "lyn": "lying",
    "lysm": "love you so much",
    "m": "male",
    "mcd": "mcdonald's",
    "mcds": "mcdonald's",
    "md@u": "mad at you",
    "me2": "me too",
    "meh": "whatever",
    "mf": "mother fucker",
    "mfb": "mother fucking bitch",
    "mgmt": "management",
    "mid": "middle",
    "mil": "mother-in-law",
    "min": "minute",
    "mins": "minutes",
    "mk": "okay",
    "mkay": "ok",
    "mmk": "ok",
    "mms": "multimedia messaging service",
    "mng": "manage",
    "mngr": "manager",
    "mod": "modification",
    "mofo": "mother fucking",
    "mojo": "attractive talent",
    "moss": "chill",
    "ms": "miss",
    "msg": "message",
    "mtg": "meeting",
    "mth": "month",
    "mu": "miss you",
    "mu@": "meet you at",
    "muah": "kiss",
    "mula": "money",
    "mwa": "kiss",
    "mwah": "kiss",
    "n/m": "nevermind",
    "n/m/h": "nothing much here",
    "n/r": "no reserve",
    "n00b": "newbie",
    "n1": "nice one",
    "n1c": "no one cares",
    "n2m": "not too much",
    "n2mh": "not too much here",
    "n2w": "not to worry",
    "n64": "nintendo 64",
    "n8kd": "naked",
    "nac": "not a chance",
    "nah": "no",
    "nal": "nationality",
    "narc": "tattle tale",
    "nark": "informant",
    "naw": "no",
    "nb": "not bad",
    "nbd": "no big deal",
    "nbjf": "no brag, just fact",
    "nd": "and",
    "ne": "any",
    "ne1": "anyone",
    "ne1er": "anyone here",
    "neh": "no",
    "nemore": "anymore",
    "neva": "never",
    "neway": "anyway",
    "newaze": "anyways",
    "newb": "newbie",
    "nite": "night",
    "nn2r": "no need to reply",
    "nnito": "not necessarily in that order",
    "nnto": "no need to open",
    "nntr": "no need to reply",
    "no1": "no one",
    "noob": "newbie",
    "nooblet": "young newbie",
    "nooblord": "ultimate newbie",
    "notch": "minecraft creator",
    "nottie": "unattractive person",
    "np": "no problem",
    "nub": "newbie",
    "nuff": "enough",
    "nufn": "nothing",
    "num": "tasty",
    "nvm": "nevermind",
    "nvr": "never",
    "nvrm": "nevermind",
    "nw": "no way",
    "nxt": "next",
    "o4u": "only for you",
    "obtw": "oh, by the way",
    "obv": "obviously",
    "obvi": "obviously",
    "oc": "of course",
    "ohemgee": "oh my gosh",
    "oic": "oh, i see",
    "oicn": "oh, i see now",
    "oiy": "hi",
    "omg": "oh my god",
    "onl": "online",
    "onoz": "oh no",
    "orly": "oh really",
    "otay": "okay",
    "otw": "on the way",
    "outta": "out of",
    "ovie": "overlord",
    "ownage": "completely owned",
    "p/d": "per day",
    "p/m": "per month",
    "p/y": "per year",
    "p911": "parent alert!",
    "p@h": "parents at home",
    "pc": "personal computer",
    "pda": "public display of affection",
    "pic": "picture",
    "pj": "poor joke",
    "pl8": "plate",
    "pld": "played",
    "pls": "please",
    "plz": "please",
    "plzrd": "please read",
    "pov": "point of view",
    "ppl": "people",
    "ppp": "peace",
    "prof": "professor",
    "prolly": "probably",
    "promo": "promotion",
    "props": "recognition",
    "prot": "protection",
    "prvt": "private",
    "ps": "postscript",
    "ps2": "playstation 2",
    "ps3": "playstation 3",
    "psa": "public service announcement",
    "psog": "pure stroke of genius",
    "psp": "playstation portable",
    "ptm": "please tell me",
    "pwd": "password",
    "psd": "password",
    "pswd": "password",
    "pwnd": "owned",
    "pwned": "owned",
    "pwnt": "owned",
    "q4u": "question for you",
    "qfe": "quoted for emphasis",
    "qft": "quoted for truth",
    "qq": "quick question",
    "qqn": "looking",
    "qrg": "quick reference guide",
    "qt": "cutie",
    "qtpi": "cutie pie",
    "r": "are",
    "r8": "rate",
    "rdy": "ready",
    "re": "replay",
    "rehi": "hi again",
    "rents": "parents",
    "rep": "reputation",
    "resq": "rescue",
    "rgd": "regard",
    "rgds": "regards",
    "ridic": "ridiculous",
    "rip": "rest in peace",
    "rl": "real life",
    "rlrt": "real life retweet",
    "rly": "really",
    "rm": "room",
    "rn": "run",
    "rnt": "aren't",
    "rof": "laughing",
    "rofl": "laughing",
    "roflmao": "laughing",
    "roflol": "laughing out loud",
    "rolf": "laughing",
    "ru": "are you",
    "ruc": "are you coming?",
    "rut": "are you there?",
    "rx": "prescription",
    "s/o": "sold out",
    "s/u": "shut up",
    "s/w": "software",
    "s2r": "send to receive",
    "s2s": "sorry to say",
    "s2u": "same to you",
    "samzd": "still amazed",
    "sd": "sweet dreams",
    "sec": "second",
    "sho": "sure",
    "sh^": "shut up",
    "siul8r": "see you later",
    "siv": "bad goaltender",
    "sk8": "skate",
    "sk8r": "skater",
    "sly": "still love you",
    "smf": "so much fun",
    "smooch": "kiss",
    "sorta": "sort of",
    "spec": "specialization",
    "spk": "speak",
    "spkr": "speaker",
    "srry": "sorry",
    "srs": "serious",
    "srsly": "seriously",
    "sry": "sorry",
    "stpd": "stupid",
    "str": "strength",
    "str8": "straight",
    "sup": "what's up",
    "syl": "see you later",
    "sync": "synchronize",
    "t2go": "time to go",
    "t2m": "talk to me",
    "t2u": "talk to you",
    "t2ul": "talk to you later",
    "t2ul8er": "talk to you later",
    "t2ul8r": "talk to you later",
    "t4lmk": "thanks for letting me know",
    "t4p": "thanks for posting",
    "t4t": "thanks for trade",
    "tc": "take care",
    "teh": "the",
    "teme": "tell me",
    "tg": "thank goodness",
    "thnq": "thank you",
    "tho": "though",
    "thru": "through",
    "tht": "that",
    "thx": "thanks",
    "tl": "tell",
    "tlk": "talk",
    "tlkin": "talking",
    "tlking": "talking",
    "tomoz": "tomorrow",
    "tq": "thank you",
    "tqvm": "thank you very much",
    "tru": "true",
    "ttl": "talk to you later",
    "ttly": "totally",
    "ttul": "talk to you later",
    "tty": "talk to you",
    "tu": "thank you",
    "tude": "attitude",
    "tx": "thanks",
    "txt": "text",
    "txtin": "texting",
    "ty": "thank you",
    "tyfa": "thank you for asking",
    "tyl": "thank you lord",
    "tym": "thank you much",
    "tyt": "take your time",
    "tyvm": "thank you very much",
    "u": "you",
    "u-ok": "you ok?",
    "u/l": "upload",
    "u2": "you too",
    "u2u": "up to you",
    "uok": "you ok?",
    "ur": "your",
    "ut": "you there?",
    "veggies": "vegetables",
    "vry": "very",
    "vs": "versus",
    "w/": "with",
    "w/b": "welcome back",
    "w/e": "whatever",
    "w/o": "without",
    "w2f": "way too funny",
    "w2g": "way to go",
    "w2k": "windows 2000",
    "w4u": "wait for you",
    "w8": "wait",
    "w84m": "wait for me",
    "w8am": "wait a minute",
    "w8ing": "waiting",
    "w8n": "waiting",
    "wa": "what",
    "waa": "crying",
    "wack": "strange",
    "wan2": "want to",
    "wannabe": "want to be",
    "wat": "what",
    "watev": "whatever",
    "watevs": "whatever",
    "wlcm": "welcome",
    "wha": "what",
    "whipped": "tired",
    "wht": "what",
    "wk": "week",
    "wknd": "weekend",
    "wtf": "what the fuck",
    "wtg": "way to go",
    "wup": "what's up?",
    "ya": "yes",
    "yeap": "yes",
    "yep": "yes",
    "yepperz": "yes",
    "yesh": "yes",
    "yo": "hi",
    "yr": "your",
    "yrs": "years",
    "yt": "you there?",
    "yt?": "you there?",
    "yup": "yes",
    "yupz": "ok",
    "zzz": "sleeping",
}

In [ ]:
appos_dict = {
    "can't": "cannot",
    "cant": "cannot",
    "aren't": "are not",
    "arent": "are not",
    "couldn't": "could not",
    "couldnt": "could not",
    "doesn't": "does not",
    "doesnt": "does not",
    "don't": "do not",
    "dont": "do not",
    "hadn't": "had not",
    "hadnt": "had not",
    "hasn't": "has not",
    "hasnt": "has not",
    "haven't": "have not",
    "havent": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "i'd": "I would",
    "i'll": "I will",
    "i'm": "I am",
    "im": "I am",
    "isn't": "is not",
    "isnt": "is not",
    "it's": "it is",
    "it'll": "it will",
    "i've": "I have",
    "let's": "let us",
    "mightn't": "might not",
    "mightnt": "might not",
    "mustn't": "must not",
    "mustnt": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "shouldn't": "should not",
    "shouldnt": "should not",
    "that's": "that is",
    "thats": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "weren't": "were not",
    "we've": "we have",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "whats": "what is",
    "what've": "what have",
    "where's": "where is",
    "who'd": "who would",
    "who'll": "who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "won't": "will not",
    "wont": "will not",
    "wouldn't": "would not",
    "wouldnt": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
    "wasn't": "was not",
    "wasnt": "was not",
    "we'll": "will",
    "didn't": "did not",
    "didnt": "did not"
}

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
emo = {
"O(<>'<>)O ":0,
'==(:-)=' :0,
'=|:)>' :0,
'=|:-)' :0,
'=|:-)=' :0,
'=(_)' :-1,
':%)' :0,
'?:^[]' :0,
':%)%' :-1,
'):=' :0,
'/.#(' :0,
':-=)' :1,
'8-[' :-1,
'(ap)' :0,
"-|--' ":0,
':-o' :-1,
'c|B-)' :0,
'(.V.)' :0,
'(8>/--<' :0,
'OOOOOO' :0,
'\^/' :1,
'(<>..<>)' :0,
'::-)' :1,
'>(::O>' :0,
'>-)' :1,
'<0>..<0>' :0,
'((-(-.-)-))' :1,
':8)' :1,
'>*^,^,^~~~' :-1,
'0=)' :1,
'^j^' :1,
'O:)' :1,
'(a)' :1,
'O:-)' :1,
'O-)' :1,
'0-)<' :1,
'0;)' :1,
'0:)' :1,
'0*-)' :1,
'():)' :1,
'J^' :1,
'(angel)' :1,
':-t' :-1,
':-||' :-1,
'X-(' :-1,
'X(' :-1,
':-@' :-1,
':@' :-1,
':(' :-1,
'>-(' :-1,
'D<' :-1,
'>:(' :-1,
'>:-(' :-1,
'-_-+' :-1,
':-Z' :-1,
'(((p(>o<)q)))' :-1,
':=@' :-1,
'x=(' :-1,
'(angry)' :-1,
'>(' :-1,
'(_)' :-1,
'(#_)' :-1,
'()' :-1,
'>:-o' :-1,
'>:-0' :-1,
'||' :-1,
'^w^' :1,
'-_-;' :0,
"|_|' ":0,
'^^;' :0,
'@_@' :-1,
'' :-1,
'>_<*' :-1,
':/' :-1,
'>:-(' :-1,
'(_")' :-1,
':42:' :0,
"-o,,o,,o' ":0,
'>:' :0,
'()' :1,
'm(_)m' :0,
'm(T-T)m' :0,
'=D>' :1,
'(^^)//' :1,
'()' :1,
'0-+' :0,
'>A<' :1,
"<')))~ ":0,
',,\(o3o)/,,' :0,
'>--->>' :0,
'<---<<<' :0,
':arrow:' :0,
'(B-I=' :-1,
'(_)' :-1,
'(_!l)' :-1,
'(_)' :-1,
'(>)' :-1,
'<|-)' :0,
'|-I' :0,
':D' : 1,
'(0)-<-<' :0,
'~X(' :1,
'X:-)' :0,
'(au)' :0,
':S' :-1,
'o.e' :-1,
':=8)' :0,
'~:0' :0,
'~O><' :0,
'[{-_-}]' :0,
'~:O' :0,
'X:-)' :-1,
'_' :-1,
'=.=' :-1,
'C:-|' :1,
'(8-)' :1,
':-)-S=~' :0,
'o~' :0,
'((' :0,
'(bandit)' :0,
'(headbang)' :0,
'(banghead)' :0,
"~'v ":0,
':-%' :0,
'~~~' :0,
':-C~' :0,
'8o|' :0,
':-E' :0,
'>--<(:p' :0,
'd:-)' :0,
'd:-p' :0,
'q:-)' :0,
'^v^' :0,
'(^+.+^)' :0,
'\()/' :0,
'B-|' :0,
'B-)' :1,
';;)' :1,
'_' :-1,
'()' :-1,
'(brb)' :1,
"('')-.-('') ":0,
"\'''\(*.*)/'''/ ":0,
'O_______' :0,
'(""\(O_O)/"")' :0,
'(()' :0,
'(())' :0,
'(())' :0,
'()' :0,
"(''')-.-(''') ":0,
'' :0,
'(())' :0,
'' :0,
'()' :0,
':)#' :1,
'(:-{~' :0,
':-)}' :0,
'b-(' :0,
'%+|' :-1,
'(#_#)' :-1,
'(*@_#)' :-1,
'(O_.)' :-1,
'(=_)' :-1,
'0___' :0,
'X:-|' :0,
'~(,)~' :0,
':bz' :0,
'(b)' :0,
'___' :0,
'|__|/' :0,
'[%]D' :0,
'(beer)' :0,
'(bricklayers)' :0,
'(_)3' :0,
'm(-_-)m' :-1,
'::::[||]::::' :0,
'_' :0,
'OTL' :-1,
'' :1,
'&:-o-8-<' :0,
'#:o+=' :0,
'o_0' :-1,
'(".")' :-1,
'|_______|' :0,
':-D' :1,
':grin:' :1,
':=D' :1,
'(laugh)' :1,
'()' :1,
'>:D<' :0,
'8|___)' :0,
':-(|)' :0,
'(|X0||)' :0,
'o=>' :-1,
'(*v*)' :0,
'<;' :0,
'=W' :0,
'~:<>' :0,
'/"' :0,
'(8>)<' :0,
'~O~' :0,
'll' :0,
'0>' :0,
'{v}' :0,
'(`)' :0,
'()' :0,
'' :0,
'()' :0,
'(^)' :0,
'(cake)' :0,
'*<:)' :1,
'<>:-)' :0,
':*x' :-1,
':-s' :0,
'(:-D' :-1,
'!-(' :0,
'?-(' :0,
'(bah)' :0,
'8^P' :-1,
'|-(' :-1,
'#-)' :-1,
'^^' :1,
':c)' :1,
'<^^*>' :1,
'(@)/(@)' :-1,
'(O)' :-1,
'(--)*' :0,
'*8=(:' :0,
':-">' :-1,
":,') ":-1,
'=">' :-1,
':-*>' :-1,
'=^}' :-1,
':")' :-1,
'=^_^=' :-1,
'=<_<=' :-1,
':*-)' :-1,
':">' :-1,
':8}' :-1,
'(blush)' :-1,
':=$' :-1,
'>//<' :-1,
'c-----7' :0,
'@=' :-1,
'(_)' :0,
'()' :0,
':-o' :-1,
':/' :-1,
'(-_-)' :-1,
'\-o' :-1,
':-o' :-1,
':z' :-1,
'(_)' :0,
'(/-_)/D>' :0,
':-)8' :1,
':-)x' :1,
'orz' :1,
'(bow)' :1,
'(||)' :0,
'===O____iii' :0,
'_____O___III' :0,
'____o/|||' :0,
'o(^_-)O' :0,
'O??' :0,
'(z)' :0,
'(:-)' :1,
':-)^<' :0,
':-#' :-1,
':-{#}' :-1,
'=#' :-1,
'%-6' :-1,
'[<O>]' :0,
'' :0,
'>:/' :0,
'%-(' :-1,
'</3' :-1,
'(U)' :-1,
'=((' :-1,
'<~3' :-1,
'<\3' :-1,
'(brokenheart)' :-1,
':-(=)' :1,
':B' :0,
'(*^o^)(^o^*)' :0,
'}|{' :-1,
'=:)' :-1,
'()_()' :-1,
'(bug)' :-1,
'E=B' :-1,
"pq`#' ":0,
'):|' :0,
'3:(:)' :0,
'[O==O\=<(' :0,
'o^o-o{' :0,
'=;-(' :-1,
'(_)' :0,
'(")(")' :0,
'(")(")' :0,
"('')_('') ":0,
'-()()-' :0,
'' :0,
'(=^_^=)' :0,
"(='.') ":0,
'(O:3' :0,
'(")_(")' :0,
'o(_-\_' :0,
'()' :0,
'U.U' :0,
'(\_/)' :0,
'#-(' :0,
'(bus)' :0,
'[===]' :0,
':-)<////>' :0,
'()' :0,
'(_|_)' :0,
'(_!_)' :0,
'}i{' :0,
'})i({' :0,
'})U({' :0,
'' :0,
'`-----------`' :0,
'(call)' :0,
':-c' :0,
"~M' ":0,
'(p)' :0,
'|C|' :0,
'|P|' :0,
'o=' :0,
'-=' :0,
'~=' :0,
'~===|' :0,
'>(///)<' :0,
'P=\\' :0,
'((-_(-_-)_-))' :0,
'(^o)(^0^)(o^)' :0,
'(cash)' :0,
'(mo)' :0,
'($)' :0,
'|*-*|' :0,
">'o'< ":0,
'=^.^=' :0,
'>;3' :0,
',,,^..^,,,~' :0,
">'.'< ":0,
'(^-_-^)' :0,
'<^oo^>' :0,
'=^_^=' :0,
'(==)' :0,
'(=^^=)' :0,
'>^..^<' :0,
'(@)' :0,
'<^..^>' :0,
'^-.-^' :-1,
'(:@@@@@D' :-1,
'<0_/\_/\_' :-1,
'~o(:-)' :0,
'(O)' :1,
'(<._.>)' :0,
'|-{' :0,
'C|:-=' :0,
'()' :0,
':-@' :-1,
'\o/' :1,
'' :1,
'' :1,
'()' :1,
'*\O/*' :1,
'C=:-)' :0,
'(:=3' :0,
'~:>' :0,
'<)' :0,
'(*>' :0,
'8^' :0,
'-{:-]' :0,
':putnam:' :0,
'*<<<<=' :0,
'' :0,
'|_|' :0,
'^)_(^' :1,
'<{^-^}>' :1,
'(ci)' :0,
'=====~' :0,
':-.)' :1,
'(:)' :0,
'(clap)' :1,
'()//' :1,
'(o)' :0,
'<]:o){' :0,
':O)' :0,
'*:o)' :0,
'<|:^0|<' :0,
'*<:o)' :0,
'*<:^)' :0,
'*<:oP' :0,
'*<:oB' :0,
'(c:>*' :0,
'*<8-)X' :0,
'><*:oDX' :0,
'' :0,
'=:-)' :0,
'(C)' :0,
'C[_]' :0,
'~0)|' :0,
'(coffee)' :0,
':-{}' :0,
'' :0,
'?_?' :0,
'$:-)' :0,
'(;)(-_-)' :1,
'(co)' :0,
'_____' :0,
'|O-O|' :0,
'|:::|' :0,
'-----' :0,
'@/:^)' :0,
':-r' :0,
':-8(' :-1,
':^)' :-1,
'O.o' :-1,
':-Q' :-1,
':-/' :-1,
':-S' :-1,
':s' :-1,
':-$' :-1,
'(@_@)' :-1,
'=?' :-1,
",':( ":-1,
'O.o?' :-1,
':-??' :-1,
'(p_q)' :-1,
'\(_o)/' :-1,
'?:|' :-1,
'-.-?' :-1,
'(-?)' :-1,
'()' :-1,
'(-_-)' :-1,
':?' :-1,
':-?' :-1,
':???:' :-1,
'o_O' :-1,
'(??)' :-1,
'8-/' :-1,
'8/' :-1,
'B-)' :1,
'qB-)' :1,
'-D' :1,
'8=)' :1,
'B=)' :1,
'(cool)' :1,
':B)' :1,
'B)' :1,
'-<0oo' :0,
'3:-O' :0,
'3:O' :0,
'3:08' :0,
'<):-)' :1,
'}:-)' :1,
'<):)' :1,
'C):)' :1,
'(-:(3' :1,
'C):-)' :1,
'{:-)' :0,
')O{)|><' :0,
'V.v.V' :-1,
'V.^.' :0,
'(\/)!_!(\/)' :-1,
'' :-1,
'()_RED_(>' :0,
'())Crayola))>' :0,
'(_)' :0,
'%-)' :0,
'()' :0,
'>,=,-' :0,
':_(' :1,
':*(' :1,
'T_T' :1,
':((' :1,
":'( ":1,
';_;' :1,
'QQ' :1,
':..[' :1,
'T^T' :1,
"='( ":1,
'8:::(' :1,
'TT^TT' :1,
'TT_TT' :1,
'i_i' :1,
":'-C ":1,
';-;' :1,
':"-(' :1,
':~-(' :1,
'TT.TT' :1,
'(!_!)' :1,
'&.(..' :1,
'(ToT)' :1,
'(;_;)' :1,
":'-( ":1,
';(' :1,
'Y.Y' :1,
'T.T' :1,
':cry:' :1,
';=(' :1,
'(cry)' :1,
'(-__-)' :1,
'|_|' :0,
'C(_)' :0,
'|_P' :0,
'C(_)' :0,
'|_P' :0,
'~o)' :0,
'[_]P' :0,
'@:-)' :0,
'8:-)' :1,
':-@!' :0,
'X3' :1,
'n_n' :1,
'||' :0,
'@-|' :0,
'()' :0,
'o-)' :0,
'O-(' :-1,
'.)' :0,
'Q-)' :0,
'*-(' :0,
'>P-(' :0,
':/7)' :0,
':)-S=' :0,
'\:D/' :1,
'<(..<)' :0,
'(>..)>' :0,
'~(o.o)~' :0,
'(^_^)(^_^)' :0,
'v(*.*)^' :0,
'(~^.^)~' :0,
'(dance)' :0,
'\o/' :0,
'(.)' :0,
'^^)' :0,
'(o)' :0,
'(o)' :0,
'C8<]' :0,
'()' :0,
'Q:|' :0,
'8->' :-1,
'*-*' :-1,
'**' :-1,
'[*_*]' :-1,
'x_x' :-1,
'x.x' :-1,
'XP' :-1,
'8-#' :-1,
'(X_X)' :-1,
'(+_+)' :-1,
'-=,o' :-1,
'(#_)' :-1,
'>T_T' :-1,
'(u_u)' :-1,
'(==)' :-1,
'()' :-1,
'~(O)|' :0,
'3:*)' :0,
'(T_T)->' :-1,
'(_)' :-1,
'^(#)_' :-1,
'(<_>)' :-1,
'._.' :-1,
'-----' :-1,
'(._.)' :-1,
'c_*' :-1,
'(;)' :-1,
':-[' :-1,
'i-)' :0,
'()' :-1,
'>:)' :-1,
'(6)' :-1,
']:->' :-1,
'}:-)' :-1,
'()}:o)' :-1,
'}=^{|~' :-1,
'3:)' :-1,
'>:-)~' :-1,
'>;->' :-1,
'3:-)' :-1,
':evil:' :-1,
'(devil)' :-1,
':[' :-1,
':-[' :-1,
'>:->' :-1,
'[](*' :0,
'{>O' :0,
":-') ":0,
'/\"' :0,
'_..._' :0,
'=|:-)X' :0,
':-|' :-1,
':|' :-1,
':(&' :-1,
'=\"' :-1,
'Disappointe' :0,
'' :-1,
'DX' :-1,
'-__-' :-1,
'(*m)' :-1,
'/:(' :-1,
'%-}' :-1,
'd[-_-]b' :0,
'[:-}' :0,
'd^_^b' :0,
':-$' :0,
':-)-o' :0,
'O:v)>--o' :0,
':O?' :0,
':O/' :0,
':O$' :0,
':O)' :0,
'<[.,=.,]-' :0,
'<[~.~]>' :0,
':o3' :0,
'<*^)' :0,
'<*^)' :0,
':3-<' :0,
'/(^.^)\"' :0,
'(UxU)' :0,
'UU' :0,
'' :0,
'U)~' :0,
'' :0,
'' :0,
'8==3' :0,
'&===3' :0,
'(&)' :0,
'VV' :0,
'(>.<)' :0,
'[$(1)$]' :-1,
'===:-D' :0,
'(8)<=' :0,
':D))' :0,
'}:-(=' :1,
'(>:^F)' :1,
'^.=.^' :-1,
'<*,^^,--' :-1,
'I-)' :1,
'*DRINK*' :0,
'\%/' :0,
'=P~' :0,
':-P' :0,
':P' :1,
'*DROOLING*' :0,
':)_' :0,
':-B' :0,
'(**)' :0,
'~~~~\o/~~~~' :-1,
':*)' :-1,
':*)?' :-1,
'%-<I>' :-1,
':#)' :-1,
'(drunk)' :-1,
'o_O' :-1,
'_|V|_' :0,
'8()' :0,
'\_O<' :0,
':-F' :0,
'|-(' :-1,
'|(' :-1,
'|=(' :-1,
'p^b' :-1,
'8)' :-1,
'Oo!' :1,
'<:-|' :-1,
'<:-)' :-1,
'()))TNT)))-x' :0,
'' :1,
'p^_^q' :0,
'::--((' :0,
':g' :0,
':K' :0,
'()' :1,
'()' :1,
'(:-|' :1,
'~~~\8-O' :0,
'\/\/\/\,8-O' :0,
':~' :0,
'~:3' :0,
'\_/' :0,
"~MC'~ ":0,
'}.' :0,
'*<:-)-:--:' :0,
'' :0,
'@=)' :0,
'2:)' :0,
'?:-p' :0,
':-[]' :0,
'5:-)' :0,
'&:-)' :0,
':-$' :0,
':$' :0,
':-[' :0,
'^^"' :0,
'(#^.^#)' :0,
':oops:' :0,
'(*/*)' :0,
'(##)' :0,
'//_^' :0,
'T-T' :0,
'(//.^)' :0,
'(//_^)' :0,
'(//_-)' :0,
'c//_+.' :0,
"c//_^' ":0,
'c/_-*' :0,
'(-_\\\)' :0,
'//_T)o.' :0,
'd(//^)b' :0,
'\/O\/O\/' :0,
'(emo)' :0,
',_' :0,
'(///.-)' :0,
'' :1,
'(*m*)' :-1,
'(v)' :-1,
'(envy)' :-1,
'(@){' :0,
'((((((^_^;)' :0,
'>:3' :0,
'(.\|/.)' :-1,
'):D}' :-1,
':{)' :-1,
'>:~>' :-1,
'>:{)>' :-1,
']}:-)>' :-1,
'>-)' :0,
'>:)' :0,
']:)' :0,
':twisted:' :0,
'(grin)' :0,
'>:D' :-1,
'()' :-1,
'>=)' :0,
'/^^\!' :-1,
':-O' :1,
'(*-*)' :1,
':o' :1,
'(^O^)' :1,
'()' :1,
'#-)' :-1,
'||' :-1,
'(;?`)' :-1,
'STO' :-1,
'(((ToT)~~' :-1,
'_(*)' :-1,
'<>' :0,
'P-)' :0,
'b-)' :0,
'' :0,
'}=D' :0,
'-@--@-' :0,
':&' :-1,
'(6.6)' :0,
'(_)' :0,
'?:-)' :1,
'o,..,o' :0,
'^,..,^' :0,
'(0)' :0,
'=^O.O^=' :0,
'O<-<=' :0,
';~[' :0,
'' :0,
'(~)' :0,
'(yn)' :0,
'X=' :0,
'()))))-x' :0,
'}-(((*>' :0,
'<")))><' :0,
'><(((">' :0,
'<.{{{><' :0,
'{}{' :0,
'<><' :0,
'<((((><' :0,
"<('O')> ":0,
'<*)))<' :0,
'|>///^<' :0,
'>)##)' :0,
'()#))<<' :0,
"<')+++< ":0,
'(~)' :0,
'p#d' :0,
'i-=<***' :0,
"!.'v ":0,
'OZ~<' :0,
'(_)' :0,
'()' :0,
'()' :0,
'()' :0,
'()' :0,
';D' :1,
';-)' :0,
'(ff)' :0,
';3' :1,
'(fm)' :0,
'@>---' :0,
'@>--;--' :0,
'<-#' :0,
"@-'- ":0,
'(f)' :0,
'(flower)' :0,
'###' :0,
'0==I==,==I===' :1,
"D'; ":0,
'O-&-<' :1,
':-!' :0,
'()' :0,
'G:-)' :0,
'=:-H' :0,
'C:#' :0,
'---E' :0,
'o^o-o|_' :0,
'|:-<' :-1,
'%%-' :0,
'88-' :0,
'Vm~' :0,
':^{=' :1,
'E:/' :0,
'[:-|I' :0,
'[8-]' :0,
'()' :-1,
'()' :-1,
'()' :-1,
'()' :-1,
":'| ":0,
':pd:' :0,
':bq:' :0,
':bd:' :0,
'(`*)' :1,
'8-|' :-1,
'()' :-1,
'8)~~*' :0,
'8)' :0,
':(?)' :0,
':()' :0,
'[II]D' :-1,
'' :-1,
'' :-1,
':-(' :-1,
':(' :-1,
':o{' :-1,
':-(' :-1,
'=P' :-1,
'X[' :-1,
':-P' :-1,
'><' :-1,
'B/' :-1,
'>.<*' :-1,
'-_-*' :-1,
':-L' :-1,
'()' :0,
'(fubar)' :0,
'|-[#]' :-1,
'B-}' :0,
'(_)' :-1,
'*:*' :-1,
':-(*)' :0,
'qB]' :-1,
'@}:-)' :0,
'=:o' :-1,
'' :1,
'3>(' :-1,
'(D:-]' :0,
'@:i' :0,
'{@:)}' :0,
'4:-)' :1,
'd=P' :-1,
'(>")>' :0,
'<(")>' :0,
'(",)' :0,
'(boo)' :0,
'()' :0,
'((()))' :0,
'8[+]' :0,
'()' :0,
'(g)' :0,
';))' :0,
'*GIGGLING*' :0,
'(giggle)' :0,
'(chuckle)' :0,
'()' :0,
'>:o===Q<' :0,
'(x)' :0,
'8:^)' :0,
'(-_-)#' :0,
'~0(**)0~' :0,
'(;`)' :1,
'' :1,
'>:[' :1,
'\~/' :0,
'\_/' :0,
'/:':-1,
'-o-o-' :0,
'8-)' :1,
'()-()' :0,
'-@--@-' :0,
'\o-o/' :0,
'(-_q)' :0,
'8)' :1,
'B-)' :1,
'B)' :0,
'(^0_0^)' :0,
'' :1,
'(nah)' :0,
'(nnh)' :0,
':-{>' :0,
'):()-' :0,
'o/' :0,
'()' :0,
'' :1,
'(`)' :1,
'(ToT)/~~~' :0,
':8]' :0,
'%:O' :0,
'8:]' :0,
'|<:-)' :0,
'L:-)' :0,
'Q:-)' :0,
'oO:)&' :0,
'' :0,
'(~};)' :0,
'$_$' :-1,
'($v$)' :-1,
'$__$' :-1,
".'! ":-1,
':->' :1,
':D' :1,
'-D' :1,
':oP' :1,
'' :1,
'(-_' :-1,
'v(*.*)^' :0,
'(^o^)' :0,
':-!' :-1,
"l:[' ":0,
'[>#<]' :-1,
'(.(.).)' :0,
'>:(' :-1,
'^i^' :1,
'>I<' :1,
'^_^;' :-1,
'(____)' :0,
'(_|||_)' :0,
'S:~)' :0,
'$8()' :0,
'*8D' :0,
'((:D' :0,
'~~8-O' :0,
'@:-)' :0,
'#:-)' :0,
'~#:-(' :-1,
'?:-)' :1,
'%-.-%' :0,
"@.'v ":0,
'{:-{)}' :-1,
'L-:' :1,
'(||]' :0,
'(%)' :0,
'(:>-<' :0,
':-|><' :0,
'(handshake)' :0,
'&:-]' :1,
'=)' :1,
':-)' :1,
':)' :1,
'^.^' :1,
'^^' :1,
'(:' :1,
'|___|' :1,
'^-^' :1,
'8D' :1,
'(^_^)' :1,
'8^)' :1,
':-}' :1,
'!_!' :1,
':)))' :1,
'(happy)' :1,
'()' :1,
'' :1,
'' :1,
'B^)' :1,
'8-D' :1,
'BD' :1,
'B^D' :1,
'8^D' :1,
'' :1,
'*<|:o)' :1,
'()' :1,
':")' :0,
'|||)' :0,
'<~8)' :-1,
'>O<' :-1,
'q=)' :0,
'xd:)' :1,
'*<:)' :0,
'+(:-)' :0,
"*<.'v ":0,
"d.'v ":0,
'\,,/(>.<)\,,/' :0,
'(({..}))' :-1,
'(|:-)' :0,
'>:^(' :-1,
'0(o.o)0' :0,
'd(-_-)b' :0,
'[:-)' :0,
"q[-'.'-]p ":0,
'dL-_-b' :0,
'<3' :1,
'<u3' :1,
'S2' :1,
'' :1,
'-(`.)->' :1,
'(h)' :1,
'(l)' :1,
'(heart)' :1,
'./\..|..|....' :0,
'()' :-1,
'\,,/' :0,
'(heidy)' :0,
'o_o' :0,
'(o)' :0,
'()!!' :0,
'?:)#' :0,
'' :0,
'(_;)_)' :-1,
'(_|' :-1,
'|)' :-1,
'|_)' :-1,
'~~' :0,
'(h5)' :0,
'o/\o' :0,
'^_^/' :0,
'(*)(*)' :0,
'^y^-/|\--Y^' :0,
'o->' :0,
':-/' :0,
'(#]' :0,
'L.' :0,
'L.' :0,
'8-)' :-1,
'(_8^(|)' :0,
'(8-(1)' :0,
'(_8^(1)' :1,
'~(_8^(|)' :1,
'o--}' :0,
'o(^^o)(o^^)o' :1,
'}:)' :1,
'O_' :0,
'[_]' :0,
']:-:)' :0,
'(h)' :0,
'(;)' :0,
'(:)\/<' :0,
':)X' :0,
"(>'')><(''<) ":0,
'(((H)))' :0,
'{}' :0,
'({})' :0,
'(bear)' :0,
'(hug)' :0,
'()' :0,
'{*}' :0,
'd=()=b' :0,
'<(^.^<)' :0,
'(>^_^)><(^o^<)' :-1,
'<(*.*<)' :0,
':D<' :1,
'((()))' :1,
'{{{***}}}' :1,
'[.]' :0,
'~!~' :-1,
'%*@:-(' :-1,
':0' :0,
':-[]' :0,
':!!' :0,
'XD' :1,
'@-)' :1,
':^)' :0,
'X_X' :0,
'_\m/' :1,
'~oO>' :1,
'<OO' :1,
'<Oo*' :1,
'<><' :0,
'*-:)' :0,
':idea:' :0,
'*help' :0,
'*red+u' :0,
':!:' :1,
'O-S-<' :0,
'(@)' :0,
'(*_*)' :1,
'<3+<3' :1,
'(_)' :1,
':]' :1,
'(inlove)' :1,
'(love)' :1,
':-]' :1,
'._.' :-1,
'_/\,' :0,
':-S' :-1,
'(o_O)' :-1,
':7/' :-1,
':-|' :-1,
':-I' :-1,
'E_E' :-1,
':neutral:' :-1,
'**' :1,
'(*)=3' :1,
'()' :1,
'0:-)' :0,
'O:D' :1,
'(wm)' :0,
'?_?' :-1,
'%7<' :-1,
'(ip)' :0,
'^#(^' :0,
'(wasntme)' :0,
':<|' :0,
'(:-|K-' :1,
'.().' :0,
'8(>_<)8' :-1,
'8-)' :1,
'//o-o\\' :0,
'((*J*))' :0,
'.-' :0,
'X-p' :1,
'(jk)' :1,
'*JOKINGLY*' :-1,
':-]' :1,
'\m/<(^_^)>\m/' :-1,
'()/' :-1,
'o(^o^)o' :-1,
'()' :-1,
'(:-#' :1,
'c",)' :0,
'K:P' :0,
'X:-)' :0,
'\VVV/' :0,
'\$$$/' :0,
']:)|--<' :0,
'<(^.^)>' :0,
"<('.'<) ":0,
"(>'.')> ":0,
'<(^_^)>' :0,
"<('.')> ":0,
"<('-'<) ":0,
"(>'')> ":0,
'c(-_-c)' :0,
'<(^,..,^)>' :0,
'(^^)' :0,
':-)*' :0,
':-X' :0,
':-*' :0,
';-(!)' :0,
':*)' :0,
'8^*' :0,
':+' :0,
':P' :1,
':*' :0,
'X(*/*)x' :0,
'(3)' :0,
'(xo)' :0,
':-{}' :0,
'(^3^)' :0,
'(*^3^)/~' :0,
'(*^^)^*)' :0,
':=*' :0,
'(kiss)' :0,
'()`)' :0,
'(_X_)' :0,
'' :0,
'*KISSED*' :0,
':}{:' :0,
'*KISSING*' :0,
'(}{)' :0,
'(-}{-)' :0,
':**:' :0,
'(=^-^=)' :0,
'=^.^=' :1,
'=+=' :1,
':3' :1,
'=^w^=' :1,
'(=^--^=)' :1,
'(=^^=)' :1,
'[|]' :0,
'>>:-|' :0,
'.-=o=-.' :1,
'+<#^v' :0,
'+<||-)' :0,
'@(*o*)@' :0,
'' :0,
'' :0,
'' :0,
':*)O' :0,
'' :0,
'&0(80<' :0,
'<{::}>' :0,
'(::)<' :0,
'-.-' :-1,
'' :0,
'l8r)' :0,
':))' :1,
'XD' :1,
':L' :1,
":'D ":1,
'=^D' :1,
'>8-D' :1,
'(^_^)' :1,
'()' :1,
'-*)' :1,
':o' :1,
'' :1,
'' :1,
'=-D' :1,
'=D' :1,
'X-D' :1,
'(lol)' :1,
':lol:' :1,
':.-)' :1,
'({)' :0,
'()' :-1,
':^o' :-1,
':PdPdP' :-1,
':-9' :-1,
'(`)' :-1,
'#-)' :1,
'(i)' :0,
'(li)' :0,
'(Y)' :1,
':3' :0,
':-*' :-1,
':-#' :-1,
':-X' :-1,
':X' :-1,
':#' :-1,
':=x' :-1,
':=#' :-1,
':()' :0,
':-{}' :0,
'd-_-b' :0,
'd^_^b' :0,
'd(^_^)b' :0,
'o(^_^)o' :0,
'd(-\\\)b' :0,
'_\\//' :1,
'>-<' :-1,
'(.`.)' :-1,
'(**)' :-1,
'~~I==I>' :0,
'' :0,
'@---' :0,
':P@---' :0,
':~)' :0,
'(*)?(*)' :-1,
'>_>' :-1,
'<_<' :-1,
'>.>' :-1,
'\\(<.<|>.>)//' :-1,
'(_)' :-1,
'<o?o>' :-1,
'<*_*>' :-1,
'L-)' :-1,
':-{}' :0,
'<3' :1,
'~~' :1,
':x' :0,
'__.' :0,
'o@=<' :0,
'=/' :0,
'>=(' :0,
'>,<' :0,
'~:(' :0,
'>=[' :0,
'>}:-(' :0,
':U' :0,
')8^(' :0,
':!' :0,
">;-(' ":0,
'>_<' :0,
':x' :0,
':-x' :0,
':mad:' :0,
'(#)' :0,
':-.)' :0,
'{8-*' :1,
'*)' :1,
'<*:-)' :-1,
'(malthe)' :0,
'0-|-[' :0,
'@@@@:-)' :1,
'(@@@@):^)' :1,
'######:o)' :1,
'@@@:)' :1,
'(#-[' :0,
':-)(-:' :0,
'' :0,
'' :0,
'-----(_)-' :-1,
'-()_)()_)' :-1,
'(.!.)' :0,
'=:)' :0,
'Q>--|' :0,
'(d)' :0,
'|>--|' :0,
'(drink)' :0,
'>:-I' :0,
'@|-)' :0,
'(-_-)' :0,
'' :0,
'' :-1,
'@_@' :1,
'\m/' :0,
'lml' :0,
'lml_' :0,
'|:-[|]' :0,
'8(:-)' :0,
'o/<' :0,
'(o:v)' :0,
':p' :1,
':}' :-1,
'?8}' :-1,
'=)=' :1,
':D:' :1,
'(mm)' :0,
'(mp)' :0,
'^o^' :0,
'-:-)' :0,
"=.'v ":0,
'(mo)' :0,
'[*(1)*]' :0,
'$-)' :0,
':-$' :0,
':-($)' :0,
'O|-)' :0,
'+<:-|' :0,
'@(-_-)@' :0,
':(|)' :0,
"@('.')@ ":0,
'---' :0,
"@'o'@ ":0,
'@(^_^)@' :0,
'@(*_*)@' :0,
'()=' :0,
'(;_)' :0,
':E' :-1,
';..;' :-1,
'VVV' :-1,
'>:(><)' :-1,
'(_i_)' :0,
'(mooning)' :0,
'(|)' :0,
'o-7-o' :0,
'o--7o' :0,
':o&o>' :0,
'^v^v^' :0,
'~<>^>' :0,
'oO~' :0,
'<:3)~' :0,
'<:3)~~' :0,
'<:3)~~~' :0,
'8(:-)' :0,
'(~)' :0,
'(film)' :0,
'(movie)' :0,
'(m)' :0,
'cl_l' :0,
'[II]D' :0,
'(_)' :0,
'(flex)' :0,
'(muscle)' :0,
'' :1,
'(music)' :0,
':-{' :0,
':{' :0,
'={D' :0,
':{D' :0,
':{)' :0,
':-#|' :0,
':-3' :0,
':{|' :0,
':-:' :-1,
'(myspace)' :0,
':_:' :0,
':-SS' :0,
'(Z(:^P' :0,
'~,~' :0,
'_:^)' :0,
':3' :0,
'*:-)' :-1,
'@:-)' :-1,
'#:-)' :-1,
'=:-)' :0,
'(-)' :0,
'*8-I' :-1,
':-B' :-1,
'8-|' :-1,
'8-B' :-1,
':-=H' :-1,
'B-|' :-1,
'8|' :-1,
'B|' :-1,
'(nerd)' :-1,
'8=|' :-1,
'B=|' :-1,
'|-(:-)' :-1,
'[-_-]~' :0,
'Qx' :0,
'QK' :0,
'(j)' :0,
'(ninja)' :0,
'[+..]' :0,
'(nod)' :0,
'(?_?)' :-1,
'):-(' :-1,
'<:@0' :0,
'(:^(' :0,
'(:+)' :0,
':/\)' :0,
':v)' :0,
'>*(' :0,
'.^v' :0,
'%-(' :0,
'<2' :1,
'[-(' :-1,
'^:)^' :1,
'(8)' :0,
']:-)' :0,
'~=[,,_,,]:3' :0,
'[^._.^]' :0,
'||' :0,
'O_o' :0,
':-j' :0,
'?:-_]' :0,
'|:-{)~' :0,
'(-o-)' :-1,
':)]' :0,
'' :0,
'' :0,
'()' :0,
':-(' :-1,
'=X' :0,
'E:-)' :0,
'~#|__/~~\o/~' :0,
'^_^' :-1,
'.^_^.' :-1,
':-)))' :1,
'^o,o^' :0,
'-(*o*)-' :0,
'(^o^)' :0,
"--''--''-- ":0,
'-"-"-' :0,
'^(OvO)^' :0,
'' :0,
':v' :0,
'(:V)' :0,
'(:\/)' :0,
'(V)=|' :1,
'(>_<)' :-1,
'(_)' :0,
'8:(' :0,
'(panda)' :0,
'' :0,
'\(>o<)/' :-1,
'\\(<.<|>.>)//' :-1,
'(>-:-)-<' :0,
'(@.@)' :-1,
'{|:{)' :0,
'<:-P' :1,
'<:o)' :1,
'<o)' :1,
'(party)' :0,
'*<(:)' :0,
'<l:0' :0,
'*<(8)~/~<' :0,
'*<|8-P~' :0,
'(:V' :0,
':)>-' :1,
'(/|\)' :1,
'(^o^)y' :1,
'*p*' :1,
'(-)' :1,
'(/)' :0,
'<IIEEEEEED' :0,
'0))__)>' :0,
"('v') ":0,
'<(")' :0,
'<(^)' :0,
"<('') ":0,
'8>' :0,
'*-)' :-1,
':-/' :-1,
'XP' :-1,
'<:^]' :0,
'O<-<' :0,
'>:P' :0,
'-()' :0,
':?)' :0,
'(mp)' :0,
'(ph)' :0,
'(phone)' :0,
'((()))' :0,
'/_/_/_/_/' :0,
'\___/' :0,
'----|}' :0,
'--===[}' :0,
'(~_(]=--Z()' :0,
'=8)' :0,
'o()~' :0,
':@)' :0,
':(:)' :0,
"<('@')> ":0,
':=)' :0,
'<(^oo^)>' :0,
'^(*(oo)*)^' :0,
'()' :0,
'^..^__)~' :0,
'(oo)' :0,
'/***(^o^)***\"' :0,
"/~'v'~\' ":0,
'(v)' :0,
"^_^'' ":0,
'(o^-^o)' :0,
'<(:-)' :0,
'(__)' :0,
'(_)' :0,
'' :-1,
':---)' :0,
':-------[' :0,
'b)' :0,
'P)' :0,
';PX' :0,
':ar!' :0,
'R)' :0,
'-=,q' :0,
'<%)' :0,
'(pi)' :0,
'<|' :0,
'(pizza)' :0,
'(pl)' :0,
';T' :1,
'\/' :0,
'(+[__]:)' :1,
'(^^)' :1,
'(pu)' :0,
'O-G-<' :0,
'()' :0,
'' :0,
':-I' :0,
'(*)(*O)' :1,
'' :1,
'-[[]]--[()]-' :0,
'[|:-{O' :0,
'c|:-|' :0,
'()' :-1,
'(poolparty)' :0,
'(hrv)' :0,
'~@~' :-1,
'+O=-)' :0,
'+<:-)' :0,
';-?' :0,
':<' :-1,
'>:-[' :0,
':-\"' :0,
'\o/' :1,
'O/' :1,
'_/\_' :0,
"/\'s ":0,
'[-o<' :1,
'_/\_' :1,
'_|' :1,
'(*)' :1,
'' :1,
'+:-)' :0,
'(zilmer)' :0,
'\<><>/' :0,
'O(+>' :0,
'~<:-)' :0,
'\&&&/' :0,
':}' :0,
'@(*-*)@' :0,
'x{|:)' :0,
'(+[PSP]::)' :0,
'' :-1,
'*3' :-1,
':&' :0,
'(puking)' :0,
'(^:^)' :0,
'(~~)' :0,
'*|' :-1,
'(_)' :0,
'(punch)' :0,
'(o(o' :0,
'=(-_-' :0,
'0()=' :0,
'(o)=)`)' :0,
'()=)-)' :0,
'-:-/' :-1,
'UU' :0,
"Q('.'Q) ":0,
'()' :-1,
'C):-' :0,
'C):-' :0,
'C):-' :0,
'C):-O' :1,
'\%%%/' :0,
':?:' :0,
'>-=]' :0,
"('.') ":0,
'(")_(")' :0,
'(uu)' :0,
'8:3' :0,
'`=o=^o>' :0,
'CD:-)>' :0,
'~~:-(' :0,
'(rain)' :0,
'(london)' :0,
'(R)' :0,
'/:)' :0,
'd:o)' :0,
'=:7)~~' :0,
'=:7(~~' :0,
'(((((:-{=' :-1,
'(l)' :0,
'(k)' :0,
'(f)' :0,
'}---:o>' :0,
'o(^)o' :-1,
'~o)' :-1,
'>=D' :-1,
'?$?:-)' :1,
'(})' :1,
'/O' :0,
'(_)' :0,
'________' :0,
'*<l:o)>' :-1,
'([(' :0,
'[*.*]' :0,
':|]' :0,
':]' :0,
':]]' :0,
'[:]' :0,
'<(-_-)>' :0,
'~(o_o)~' :0,
'[:|]' :0,
'\m/(**)\m/' :0,
'\m/@_@\m/' :0,
'\m/>.<\m/' :0,
'\m/(><)\m/' :0,
'd(>_<)b' :0,
'\m/(>.<)\m/' :0,
'bd(OoO)bd' :0,
'<:-)<<|' :0,
'<===~~' :0,
'\w/^^,\w/' :0,
'>-D' :0,
'\m/' :0,
'\\m/(=o+)\m//' :0,
'(rock)' :0,
'\m/(*o*)\m/' :0,
'\m/d(^_^)b\m/' :0,
'8-|' :0,
'>>' :0,
'9_9' :0,
'@@' :0,
'c.c' :0,
':roll:' :0,
'' :0,
'=))' :1,
'(rotfl)' :1,
'(rofl)' :1,
'()' :1,
'7:-)' :0,
'\:)' :0,
'~:>' :0,
'^:3' :0,
'@)->-' :0,
'@~)~~~~' :0,
'@}->--' :0,
'@};-' :0,
"@~'~~~ ":0,
'@>-;--' :0,
'@-->---' :0,
'--{--@' :0,
'@>--' :0,
'@}-^--^--' :0,
'(@)->->->--' :0,
'@}-(' :0,
'<(*_*)>' :0,
'3:*>' :0,
'(;_)' :0,
'=()' :0,
'' :0,
'=(;)' :0,
'=(' :-1,
':-(' :-1,
':(' :-1,
'T.T' :-1,
'=[' :-1,
':c' :-1,
'D:' :-1,
'D=' :-1,
'Y_Y' :-1,
'v.v' :-1,
'\8^1' :-1,
'(<_>)' :-1,
'(/_\)' :-1,
'):' :-1,
'(_)' :-1,
':[' :-1,
':sad:' :-1,
':=(' :-1,
'(sad)' :-1,
'8-(' :-1,
'/:-)' :0,
';-)}<////>' :0,
'>>>>(:-)' :0,
'M:-)' :1,
'O7' :1,
':-{o' :0,
'*<]:{)' :1,
'*<(:-{)}' :1,
'*<:-)' :1,
'*<|:-{)}' :1,
'<|:)}' :1,
'0<[]:-)>>' :1,
'*-={:-)}}}>' :1,
'*<|:o)>' :1,
'<[]:o{|>' :1,
'*<[]:o{|>' :1,
':-7' :-1,
'^o)' :-1,
';s' :-1,
':S' :-1,
'D:' :-1,
':!' :-1,
'(;)' :-1,
'.()' :-1,
'()' :0,
'()' :-1,
'' :1,
'*!#*!^*&:-)' :-1,
'!:-)' :0,
'*s*' :0,
'8<' :0,
'c--I-I-I{' :-1,
'3#`' :-1,
'>-$' :-1,
'^:-/' :0,
':-@' :-1,
'..._[:]' :0,
'..._(:)-o' :0,
'O-)' :0,
'(:)-)' :0,
"\';./ ":0,
'' :0,
'()' :0,
';^!' :0,
':-*' :1,
'8==8' :-1,
'\000/' :0,
'(shake)' :1,
'9(x.x)9' :1,
'(x.x)9' :1,
'[-x' :-1,
'<(((">' :0,
'(^^^)' :0,
'~~~/\\~~~' :0,
'>---^-*<' :0,
'_/\____\0/__' :0,
'_/\_\o/_' :-1,
'=O' :-1,
'*SHOCKED*' :-1,
'o_o' :-1,
':-o' :-1,
'8-0' :-1,
'8^{}' :-1,
'(O_O)' :-1,
'8O' :-1,
':shock:' :-1,
'(**)' :-1,
'' :-1,
'=(.)=r' :0,
'()o*--' :0,
'()*' :0,
'' :0,
'^^' :-1,
':-V' :-1,
':\/' :-1,
'|----D' :0,
"`\('_')/` ":-1,
'\(_)/' :0,
'\_()_/' :0,
'(*^_^*)' :0,
'-_-' :0,
'(^^;)' :0,
'(#^.^#)' :0,
'(#/\#)' :0,
'(*)' :0,
'(-:|:-)' :0,
':-&' :-1,
'+o(' :-1,
'|-~b' :-1,
'(-__-)' :-1,
':-~)' :-1,
':->|' :-1,
':-Y' :0,
':-<' :0,
'8-}' :0,
'=P' :1,
'C(:' :0,
'(^o^)' :0,
':-o' :0,
'(o)~' :0,
']:' :0,
'O-\-<]:' :0,
'0-<-<):' :0,
'o{-<]:' :0,
'O[-<]:' :0,
'o<[^(' :0,
'o->-</:' :0,
'q=)-<--<|:' :0,
'O-/-<]:' :0,
':-/' :-1,
':-7' :-1,
':/' :-1,
'(.)' :-1,
'*(H-)' :0,
'=(^-^)' :0,
'8-X' :0,
'(skype)' :0,
'(ss)' :0,
'(^^)(o)' :0,
'(>.<)' :-1,
'-_-' :-1,
'-_-' :-1,
'(-.-)zzZZ' :-1,
'Z/' :-1,
'Z_Z' :-1,
'(~_~)^' :-1,
'(-_o)zzZ' :-1,
'O-(==<' :-1,
'<-.->' :-1,
'(--)zzz...' :-1,
'' :-1,
'|-)' :0,
'(>.<)' :0,
'u_u' :0,
'#)' :0,
'"8^y' :0,
'~_~' :0,
'(-_-)Zzz' :0,
'(Z.Z)' :0,
'I-)' :0,
'(snooze)' :0,
'I=)' :0,
"#(,'%/) ":0,
':-#' :0,
':-)' :1,
':)' :1,
'\_/' :1,
'[______]' :1,
':|)' :1,
'C:' :1,
'\\)' :1,
':A' :1,
':)' :1,
'=]' :1,
'\__/' :1,
'(*8' :1,
'I:(' :1,
'>-)' :1,
':^D' :1,
':]' :1,
':smile:' :1,
':=)' :1,
'(smile)' :1,
'()' :1,
':^J' :1,
'^^' :1,
'^&^' :1,
".'J ":1,
':-1' :0,
':-,' :0,
':^,' :0,
';^)' :0,
'(smirk)' :0,
'()' :1,
':-p~' :0,
':-d~' :0,
':-Q' :0,
':O\"' :0,
':-' :0,
':-?' :0,
':-d' :0,
':-G-' :0,
'(--)y-' :0,
'(())y-' :0,
'(smoke)' :0,
'(smoking)' :0,
'(ci)' :0,
'o()y' :0,
':-?' :0,
'(`)' :1,
':>' :-1,
'B|' :-1,
'(sn)' :0,
'@x' :0,
'_@/' :0,
'`@____' :0,
'@_v' :0,
'@_' :0,
'_@_v' :0,
'_@v' :0,
'~<:<<<<<>' :0,
'~~~~8}' :0,
'~>)' :0,
':-`|' :0,
':<)' :-1,
'(_' :0,
'|^O' :0,
'^o' :0,
'' :0,
'(:^)_)_)' :0,
'[l:^)' :0,
'(:-)()' :0,
'*|:^)(.)(...)' :0,
'(`)' :-1,
'(A`)' :-1,
'.`(><)`.' :-1,
'(so)' :0,
'(____)' :0,
'/:-)' :0,
'c]:{D' :0,
'' :0,
'?-(' :-1,
'=^{' :-1,
'>.<' :-1,
':-*' :-1,
':-6' :-1,
'(X^(*' :-1,
'=.=' :-1,
'[>==]' :0,
'0_o' :-1,
'|-=[O]=-|' :0,
':-M' :-1,
'=#' :-1,
':|' :-1,
':-|' :-1,
':=|' :-1,
'(speechless)' :-1,
'||||' :0,
'/\(00)/\"' :0,
'' :0,
'^;;^' :0,
'::::(' :0,
':-`' :0,
'>:-I' :0,
'===()' :0,
'__m_OO_m__' :0,
'<:' :0,
'' :0,
'(><)' :1,
'-_-' :1,
'(*)' :0,
'(star)' :0,
'*~*' :0,
'' :1,
'(;_)' :1,
'()' :1,
'%.)' :1,
'=-O' :0,
'(~);}' :0,
';@' :-1,
'||=||' :0,
'|:-|' :-1,
'' :0,
'/\"' :0,
'' :0,
'#:-)-{--<' :0,
'>-/-@' :0,
'O-[--<' :0,
':-)~' :1,
':-P' :1,
':P' :1,
'>:P' :1,
':-r' :1,
':pp' :1,
':(,)' :1,
'(:-p)' :1,
'' :1,
".'r ":1,
':razz:' :1,
':=P' :1,
'(tongueout)' :1,
'~/~' :1,
'$:-$' :0,
'>;D' :0,
'*STOP*' :0,
'(st)' :-1,
':|' :0,
".'T ":0,
'-.-' :-1,
'O=[O{|___|]=O' :0,
'@?@' :-1,
'[o].[o]' :-1,
'&;-P' :1,
'(sun)' :0,
'8-)' :0,
'B-)' :0,
'B-D' :0,
']-I' :0,
'B:-)' :0,
'8-|' :0,
'8|' :0,
'B-|' :0,
'B|' :0,
'8)' :0,
':cool:' :0,
'c-oo' :0,
'_\|/_' :1,
'(#)' :1,
'((o))' :1,
':o' :0,
':-O' :0,
'*SURPRISED*' :0,
':O' :0,
'{:o' :0,
'(:)o)' :1,
'O-' :0,
':-()' :0,
'=0' :0,
'(*.*)' :0,
'l8^O' :0,
'****' :0,
'' :0,
'*_*' :0,
':eek:' :0,
':=o' :0,
'(surprised)' :0,
'*.*' :0,
'(O.O)' :0,
'=^o' :0,
'=^0' :0,
'' :-1,
'' :-1,
'' :-1,
':#' :-1,
'>:-@!' :-1,
'(swear)' :-1,
'`:-)' :0,
'(:|' :0,
'(sweat)' :0,
'8-)' :0,
'__/\o_' :0,
'B-)-[<' :0,
'x_O' :-1,
'/________|___|' :0,
'oxx:{}:::::::>' :0,
'cxxx{}:::::::>' :0,
'0]xxxx]====>' :0,
'<=======|==o' :0,
'(>_)(_<)' :0,
'---(|||]==[]' :0,
'+=>-' :0,
'/(..)' :0,
'(>o<)' :0,
'=;' :0,
':^y' :-1,
':y' :-1,
'(talk)' :-1,
'(____)' :0,
'|"""\-=' :0,
'(____)' :0,
'>8-O-(&)' :1,
'()' :1,
'(tuari)' :0,
'\_/?' :0,
'C(_)~' :0,
"C(_)~' ":0,
':p' :1,
'B^P' :-1,
':^P' :-1,
'8^P' :-1,
':O)' :0,
'()' :0,
'(t)' :0,
'(((((^_^)' :0,
'-(:)(0)=8' :0,
'>[I' :0,
'ixxxxxi' :0,
'|B-)' :0,
'-:}' :0,
'8=:-K' :0,
':-?' :0,
'*-)' :0,
':?' :0,
':=?' :0,
'(think)' :0,
'(*_)*' :0,
'(n)' :0,
"q('-')p ":0,
':-q' :0,
'(no)' :0,
'(y)' :0,
"d'-' ":0,
"d'-'b ":0,
':-bd' :0,
'(^-^)b' :0,
'd-(^.^)z' :0,
'(ok)' :0,
'(yes)' :0,
':-)<////>' :0,
'|-(o)-|' :0,
'(-o-)' :0,
'|o|' :0,
':-T' :1,
'[]---|<O---[]' :0,
'(o)' :0,
'(time)' :0,
'(clock)' :0,
':-t' :0,
'-<:-(' :0,
'(:|' :0,
'*TIRED*' :0,
'=_=' :0,
'%-|' :0,
'%-/' :0,
'(z_z~.)' :0,
'()' :0,
'(=_=)' :0,
'(%]:D' :1,
'0000(0)(0)0000' :0,
'.oooO' :0,
'Oooo.' :0,
'(toivo)' :0,
".'w ":0,
'8-;)' :0,
':-J' :0,
':j' :0,
':-&' :0,
':-a' :0,
'(tmi)' :0,
'[|:)' :0,
'#:o\:o/:o\:o/:o||' :0,
"=') ":-1,
'{(:-)' :0,
'}:-(' :0,
'{:-)' :0,
'-HHHH' :0,
'[..]' :0,
'>~<' :0,
'>:-|' :0,
':<()' :0,
'---(o__)o' :-1,
'(>_<)>' :-1,
'(^_^;)' :-1,
'<.<' :-1,
'' :0,
'oo---oo-Bo' :0,
'-=iii=<()' :1,
'(_)' :0,
'O>-O>-|-<O-<O' :0,
'X:-|' :0,
'@=)' :0,
'@:-v' :0,
'<:>==' :0,
'mm' :0,
'' :0,
'=8>-()<' :0,
'(tu)' :1,
':(' :-1,
'/Y\"' :0,
':-)8---' :0,
':-)8>' :0,
'.-=-.' :0,
'(o|o)' :0,
'(um)' :0,
'(---,' :0,
'q;|' :0,
'x:-/' :-1,
'=l:-)=' :0,
'=):-)>' :0,
':-\"' :-1,
':\"' :-1,
'/:-|' :-1,
':<' :-1,
'|:-)' :0,
'|:' :-1,
'' :-1,
'**==' :1,
">->' ":-1,
':/' :-1,
'8-\"' :-1,
'>.<' :-1,
'}:(' :-1,
'>:*' :-1,
'D:<' :-1,
'>=P' :-1,
'>-:O' :-1,
'--"' :-1,
'.-.' :0,
'P:' :1,
':-E' :0,
"(';.;') ":0,
'@,.,@' :0,
'*,..,*' :0,
'-,..,-' :0,
':-{' :0,
':f' :0,
',..,' :0,
'(o,..,o)' :0,
'(=,..,=)' :0,
':-[' :0,
':(=' :0,
'8-E' :0,
'>:-E' :0,
'>8-E' :0,
'>:-[' :0,
'8-{' :0,
'>8-{' :0,
':-[' :0,
':[' :0,
'^,..,^' :0,
'^,..,^' :0,
'~;;~' :0,
'(^-^:)' :1,
':-[' :0,
'%*}' :-1,
'=D' :1,
':-))' :1,
':-((' :-1,
'(o^_^)o' :1,
'v(^_^)v' :1,
'()' :1,
'8<' :0,
'](-[3' :0,
'(o\_!_/o)' :-1,
'(o^)/(^o)' :0,
'' :0,
'ixxxxxxx' :0,
':O=' :-1,
':-C..' :-1,
"(>'.')># ":-1,
"(>'_')># ":-1,
'#<(0.0<)' :-1,
'(o*)' :0,
'(wait)' :0,
'(((((((`)' :0,
'(_)' :0,
':->X==|' :0,
'_()' :0,
':-w' :1,
':3=' :0,
'(:{C)' :0,
':-<' :0,
'(:3=' :0,
'._)' :1,
'__/\o_' :0,
':-h' :0,
'(wave)' :0,
'(hi)' :0,
'(bye)' :0,
'~~c___' :0,
':-)--' :-1,
'(~`)' :-1,
';w;' :-1,
'(;`)' :-1,
'(><)' :-1,
':-P' :0,
'-_-' :0,
':-1' :0,
'(_)' :0,
'(_)' :0,
'd-:' :0,
'#:-S' :0,
'(whew)' :0,
":'( ":-1,
":'-( ":-1,
'*WHINING*' :-1,
'(`)' :-1,
':-"' :0,
".'Y ":0,
'()' :0,
'()' :0,
'(w)' :-1,
'' :0,
'())=(' :0,
';p' :1,
';-P' :1,
';^)' :1,
'(`_^)' :1,
',-)' :1,
'(^_~)' :1,
'9-)' :1,
'6-)' :1,
'i)' :1,
'!)' :1,
';-)' :1,
';)' :1,
'~_^' :1,
';()' :1,
':wink:' :1,
';=)' :1,
'(wink)' :1,
'' :1,
'(^)' :1,
'!-)' :1,
'(-!' :1,
'^_^a' :-1,
'' :-1,
'<):^/' :-1,
'___' :-1,
'<|:~,' :-1,
'I8${}' :-1,
'8<:-)' :-1,
'<{{{};->~' :-1,
'<*(:-?' :-1,
'(-).*' :-1,
'{0-|<[' :0,
'o>-<|=' :0,
'o.o?' :1,
':^)' :1,
'(wonder)' :1,
'||' :0,
'(wo)' :0,
':-S' :-1,
'(;)' :-1,
'>w<' :-1,
':S' :-1,
':=S' :-1,
'(worry)' :-1,
'' :-1,
'(`)' :-1,
'()' :-1,
'()' :0,
'(_;)' :0,
'()' :0,
'(xx)' :0,
'[x]' :0,
'_/)' :0,
'|-O' :-1,
'(:|' :-1,
".'U ":-1,
'(yawn)' :-1,
'(;`)' :-1,
'*\o/*' :0,
'>:O' :-1,
'=^Y' :-1,
':-(0)' :-1,
':(*)' :-1,
'(*O)' :-1,
'(__)' :0,
'(%)' :0,
'\(o/' :0,
'<(-_-)>' :0,
'<(-.-)>' :0,
':-0>' :0,
'()' :1,
':#' :0,
'F(x_x)F' :-1,
'8-#' :-1,
'[-]' :-1,
'B-)===>' :0
}

In [ ]:
emoticons = {
"O(<>'<>)O" : "Aang",
'==(:-)=' : 'Abraham Lincoln', 
'=|:)>' : 'Abraham Lincoln', 
'=|:-)' : 'Abraham Lincoln', 
'=|:-)=' : 'Abraham Lincoln', 
'=(_)' : 'Accidental farting', 
':%)' : 'Accountant', 
'?:^[]' : 'Ace Ventura', 
':%)%' : 'Acne', 
'):=' : 'Adolf Hitler', 
'/.#(' : 'Adolf Hitler', 
':-=)' : 'Adolf Hitler', 
'8-[' : 'Afraid', 
'(ap)' : 'Airplane', 
"-|--'" : "Airplane",
':-o' : 'Alarmed', 
'c|B-)' : 'Ali G', 
'(.V.)' : 'Alien', 
'(8>/--<' : 'Alien', 
'OOOOOO' : 'Alien', 
'\^/' : 'Alien', 
'(<>..<>)' : 'Alien', 
'::-)' : 'Alien', 
"'.'" : "Alien",
'>(::O>' : 'Alien', 
'>-)' : 'Alien', 
'<0>..<0>' : 'Alien', 
'((-(-.-)-))' : 'Alien', 
':8)' : 'Alien', 
'>*^,^,^~~~' : 'Alligator', 
'0=)' : 'Angel', 
'^j^' : 'Angel', 
'O:)' : 'Angel', 
'(a)' : 'Angel', 
'O:-)' : 'Angel', 
'O-)' : 'Angel', 
'0-)<' : 'Angel', 
'0;)' : 'Angel', 
'0:)' : 'Angel', 
'0*-)' : 'Angel', 
'():)' : 'Angel', 
'J^' : 'Angel', 
'(angel)' : 'Angel', 
':-t' : 'Angry', 
':-||' : 'Angry', 
'X-(' : 'Angry', 
'X(' : 'Angry', 
':-@' : 'Angry', 
':@' : 'Angry', 
':(' : 'Angry', 
'>-(' : 'Angry', 
'O' : 'Angry', 
'D<' : 'Angry', 
'>:(' : 'Angry', 
'>:-(' : 'Angry', 
'-_-+' : 'Angry', 
':-Z' : 'Angry', 
'(((p(>o<)q)))' : 'Angry', 
':=@' : 'Angry', 
'x=(' : 'Angry', 
'(angry)' : 'Angry', 
'>(' : 'Angry', 
'(_)' : 'Angry', 
'(#_)' : 'Angry', 
'()' : 'Angry', 
'>:-o' : 'Angry', 
'>:-0' : 'Angry', 
'||' : 'Angry', 
'^w^' : 'Anime Smile', 
'-_-;' : 'Anime Sweat Drop', 
"|_|'" : "Anime Sweat Drop",
'^^;' : 'Anime Sweat Drop', 
'@_@' : 'Annoyed', 
'' : 'Annoyed', 
'>_<*' : 'Annoyed', 
':/' : 'Annoyed', 
'>:-(' : 'Annoyed', 
'(_")' : 'Annoyed', 
':42:' : 'Answer to Everything', 
"-o,,o,,o'" : "Ant",
'>:' : 'o Antelope', 
'()' : 'Anticipation', 
'm(_)m' : 'Apology', 
'm(T-T)m' : 'Apology', 
'=D>' : 'Applause', 
'(^^)//' : 'Applause', 
'()' : 'Applause', 
'0-+' : 'April', 
'>A<' : 'Archangel', 
"<')))~" : "Armadillo",
',,\(o3o)/,,' : 'Armpit Hair', 
'>--->>' : 'Arrow', 
'<---<<<' : 'Arrow', 
':arrow:' : 'Arrow', 
'(B-I=' : 'Art Critic', 
'(_)' : 'Ashamed', 
'(_!l)' : 'Ashamed', 
'(_)' : 'Ashamed', 
'(>)' : 'Ashamed', 
'<|-)' : 'Asian', 
'|-I' : 'Asleep', 
'(0)-<-<' : 'Astronaut', 
'~X(' : 'At Wits End', 
'X:-)' : 'Aunt Jemima', 
'(au)' : 'Auto', 
':S' : 'Awkward', 
'o.e' : 'Awkward', 
'Emoticons' : 'that start with B', 
'Emoticon' : 'Meaning', 
':=8)' : 'Baboon', 
'~:0' : 'Baby', 
'~O><' : 'Baby', 
'[{-_-}]' : 'Baby', 
'~:O' : 'Baby', 
'X:-)' : 'Bad hair day', 
'_' : 'Bad hair day', 
'=.=' : 'Bad Idea', 
'C:-|' : 'Bald', 
'(8-)' : 'Bald', 
':-)-S=~' : 'Ballerina', 
'o~' : 'Balloon', 
'((' : 'Banana', 
':' : 'Bandit', 
'(bandit)' : 'Bandit', 
'(headbang)' : 'Banging Head on Wall', 
'(banghead)' : 'Banging Head on Wall', 
"~'v" : "Bangs",
':-%' : 'Banker', 
'~~~' : 'Banksy', 
':-C~' : 'Barfing', 
'8o|' : 'Baring Teeth', 
':-E' : 'Baring Teeth', 
'>--<(:p' : 'Baseball Hat', 
'd:-)' : 'Baseball Player', 
'd:-p' : 'Baseball Player', 
'q:-)' : 'Baseball Player', 
'^v^' : 'Bat', 
'(^+.+^)' : 'Bat', 
'\()/' : 'Bathtub', 
'B-|' : 'Batman', 
'B-)' : 'Batman', 
';;)' : 'Batting Eyelashes', 
'_' : 'Bawling', 
'()' : 'Bawling', 
'(brb)' : 'Be Right Back', 
"('')-.-('')" : "Bear",
"\'''\(*.*)/'''/" : "Bear",
'o' : 'o', 
'O_______' : 'O', 
'(""\(O_O)/"")' : 'Bear', 
'(()' : 'Bear', 
'(())' : 'Bear', 
'(())' : 'Bear', 
'()' : 'Bear', 
"(''')-.-(''')" : "Bear",
'' : 'Bear', 
'(())' : 'Bear', 
'' : 'Bear cub', 
'()' : 'Bear cub', 
':)#' : 'Beard', 
'(:-{~' : 'Beard', 
':-)}' : 'Beard', 
'b-(' : 'Beat Up', 
'%+|' : 'Beaten Up', 
'(#_#)' : 'Beaten Up', 
'(*@_#)' : 'Beaten Up', 
'(O_.)' : 'Beaten Up', 
'(=_)' : 'Beaten Up', 
'0___' : 'Bed', 
'X:-|' : 'Bed Head', 
'~(,)~' : 'Bee', 
':bz' : 'Bee', 
'(b)' : 'Beer Mug', 
'___' : '', 
'|__|/' : 'Beer Mug', 
'[%]D' : 'Beer Mug', 
'(beer)' : 'Beer Mug', 
'(bricklayers)' : 'Beer Mug', 
'(_)3' : 'Beer Mug', 
'm(-_-)m' : 'Begging', 
'::::[||]::::' : 'Belt', 
'_' : 'Benjamin Franklin', 
'OTL' : 'Bent Over', 
'' : 'Best Buds', 
'&:-o-8-<' : 'Betty Boop', 
'#:o+=' : 'Betty Boop', 
'o_0' : 'Bewildered', 
'(".")' : 'Bewildered', 
'|_______|' : 'Bible', 
':D' : 'Big Grin', 
':-D' : 'Big Grin', 
':grin:' : 'Big Grin', 
':=D' : 'Big Grin', 
'(laugh)' : 'Big Grin', 
'()' : 'Big Grin', 
'>:D<' : 'Big Hug', 
'8|___)' : 'Big Laugh', 
':-(|)' : 'Big Lips', 
'(|X0||)' : 'Big Mac', 
'o=>' : 'Billy', 
'(*v*)' : 'Bird', 
'<;' : 'O', 
'(' : 'Bird', 
'=W' : 'Bird', 
'~:<>' : 'Bird', 
'/"' : 'Bird', 
'V' : 'Bird', 
'(8>)<' : 'Bird', 
'~O~' : 'Bird', 
'll' : 'Bird', 
'0>' : 'Bird', 
'{v}' : 'Bird', 
'(`)' : 'Bird', 
'()' : 'Bird', 
'' : 'Bird', 
'()' : 'Bird', 
'(^)' : 'Birthday Cake', 
'(cake)' : 'Birthday Cake', 
'*<:)' : 'Birthday Hat', 
'<>:-)' : 'Bishop', 
':*x' : 'Biting', 
':-s' : 'Bizzare Comment', 
'(:-D' : 'Blabber Mouth', 
'!-(' : 'Black Eye', 
'?-(' : 'Black Eye', 
'(bah)' : 'Black Sheep', 
"'8^P" : "Blah",
'|-(' : 'Blind', 
'#-)' : 'Blinking', 
'^^' : 'Blissful', 
':c)' : 'Blissful', 
'<^^*>' : 'Blissful', 
'(@)/(@)' : 'Bloodshot Eyes', 
'(O)' : 'Blowfish', 
'(--)*' : 'Blowing a Kiss', 
'*8=(:' : 'Blubbering', 
':-">' : 'Blushing', 
":,')" : "Blushing",
'=">' : 'Blushing', 
':-*>' : 'Blushing', 
'=^}' : 'Blushing', 
':")' : 'Blushing', 
'=^_^=' : 'Blushing', 
'=<_<=' : 'Blushing', 
':*-)' : 'Blushing', 
':">' : 'Blushing', 
':8}' : 'Blushing', 
'(blush)' : 'Blushing', 
':=$' : 'Blushing', 
'>//<' : 'Blushing', 
'c-----7' : 'Boat', 
'@=' : 'Bomb', 
'*' : 'Bomb', 
'(_)' : 'Boogie down', 
'()' : 'Booing', 
':-o' : 'Bored', 
':/' : 'Bored', 
'(-_-)' : 'Bored', 
'\-o' : 'Bored', 
':-o' : 'zz Bored', 
':z' : 'Bored', 
'(_)' : 'Borg', 
'(/-_)/D>' : 'Bow and Arrow', 
':-)8' : 'Bow Tie', 
':-)x' : 'Bow Tie', 
'orz' : 'Bowing', 
'(bow)' : 'Bowing', 
'(||)' : 'Bowl', 
'===O____iii' : 'Bowling', 
'_____O___III' : 'Bowling', 
'____o/|||' : 'Bowling', 
'o(^_-)O' : 'Boxer', 
'O??' : 'Boxer', 
'(z)' : 'Boy', 
'(:-)' : 'Boy', 
':-)^<' : 'Boy', 
':-#' : 'Braces', 
':-{#}' : 'Braces', 
'=#' : 'Braces', 
'%-6' : 'Brain Dead', 
'[<O>]' : 'Brazilian Flag', 
'' : 'Bread', 
'>:/' : 'Bring it On', 
'%-(' : 'Broken Glasses', 
'</3' : 'Broken Heart', 
'(U)' : 'Broken Heart', 
'=((' : 'Broken Heart', 
'<~3' : 'Broken Heart', 
'<\3' : 'Broken Heart', 
'(brokenheart)' : 'Broken Heart', 
':-(=)' : 'Buckteeth', 
':B' : 'Bucktooth', 
'(*^o^)(^o^*)' : 'Buddies', 
'}|{' : 'Bug', 
'=:)' : 'Bug', 
'()_()' : 'Bug', 
'(bug)' : 'Bug', 
'E=B' : 'Bugs Bunny', 
"pq`#'" : "Bull",
'):|' : 'Bull', 
'3:(:)' : 'Bull', 
'[O==O\=<(' : 'Bulldozer', 
'o^o-o{' : 'Bulldozer', 
'=;-(' : 'Bully', 
'(_)' : 'Bummed out', 
'(")(")' : 'Bunny', 
'(")(")' : 'Bunny', 
"('')_('')" : "Bunny",
'-()()-' : 'Bunny', 
'' : 'U U  Bunny', 
'(=^_^=)' : 'Bunny', 
"(='.')" : "Bunny",
'(O:3' : 'Bunny', 
'(")_(")' : 'Bunny', 
'o(_-\_' : 'Bunny', 
'()' : 'Bunny', 
'U.U' : 'Bunny', 
'(\_/)' : 'Bunny Ears', 
'#-(' : 'Bureaucrat', 
'(bus)' : 'Bus', 
'[===]' : 'Bus', 
':-)<////>' : 'Businessman', 
'()' : 'But At What Cost', 
'(_|_)' : 'Butt', 
'(_!_)' : 'Butt', 
'}i{' : 'Butterfly', 
'})i({' : 'Butterfly', 
'})U({' : 'Butterfly', 
'' : 'Butterfly', 
'Emoticons' : 'that start with C', 
'Emoticon' : 'Meaning', 
'`-----------`' : 'Cake', 
'(call)' : 'Call', 
':-c' : 'Call Me', 
"~M'" : "Camel",
'(p)' : 'Camera', 
'|C|' : 'Can of Coke', 
'|P|' : 'Can of Pepsi', 
'o=' : 'Candle', 
'-=' : 'Candle', 
'~=' : 'Candle', 
'~===|' : 'Candle', 
'>(///)<' : 'Candy', 
'P=\\' : 'Cap', 
'__' : '', 
'((-_(-_-)_-))' : 'Care Crowd', 
'(^o)(^0^)(o^)' : 'Caroling', 
'(cash)' : 'Cash', 
'(mo)' : 'Cash', 
'($)' : 'Cash', 
'|*-*|' : 'Cassette Tape', 
">'o'<" : "Cat",
'=^.^=' : 'Cat', 
'>;3' : 'Cat', 
',,,^..^,,,~' : 'Cat', 
'~' : 'Cat', 
">'.'<" : "Cat",
'(^-_-^)' : 'Cat', 
'<^oo^>' : 'Cat', 
'=^_^=' : 'Cat', 
'(==)' : 'Cat', 
'(=^^=)' : 'Cat', 
'>^..^<' : 'Cat Face', 
'(@)' : 'Cat Face', 
'<^..^>' : 'Cat Face', 
'^-.-^' : 'Zzzzz Cat Nap', 
'(:@@@@@D' : 'Caterpillar', 
'<0_/\_/\_' : 'Caterpillar', 
'~o(:-)' : 'Cave Diver', 
'(O)' : 'Celebrate', 
'(<._.>)' : 'Charlie Brown', 
'|-{' : 'Charlie Brown', 
'C|:-=' : 'Charlie Chaplin', 
'()' : 'Chasing A Butterfly', 
':-@' : 'Chatterbox', 
'\o/' : 'Cheer', 
'' : 'Cheerful', 
'' : 'Cheerful', 
'()' : 'Cheerful', 
'*\O/*' : 'Cheerleader', 
'C=:-)' : 'Chef', 
'(:=3' : 'Chef', 
'~:>' : 'Chicken', 
'<)' : 'Chicken', 
'(*>' : 'Chicken', 
'8^' : 'Chicken', 
'-{:-]' : 'Chinese Hat', 
':putnam:' : 'Chris Putnam', 
'*<<<<=' : 'Christmas Tree', 
'' : 'X', 
'|_|' : 'Christmas Tree', 
'|' : 'Christmas Tree', 
'^)_(^' : 'Chubby', 
'<{^-^}>' : 'Chubby Face', 
'(ci)' : 'Cigarette', 
'=====~' : 'Cigarette', 
':-.)' : 'Cindy Crawford', 
'(:)' : 'Clam', 
'(clap)' : 'Clapping', 
'()//' : 'Clapping', 
'(o)' : 'Clock', 
'<]:o){' : 'Clown', 
':O)' : 'Clown', 
'*:o)' : 'Clown', 
'<|:^0|<' : 'Clown', 
'*<:o)' : 'Clown', 
'*<:^)' : 'Clown', 
'*<:oP' : 'Clown', 
'*<:oB' : 'Clown', 
'(c:>*' : 'Clown', 
'*<8-)X' : 'Clown', 
'><*:oDX' : 'Clown', 
'' : 'Clown', 
'=:-)' : 'Cobbler', 
'(C)' : 'Coffee Cup', 
'C[_]' : 'Coffee Cup', 
'~0)|' : 'Coffee Cup', 
'(coffee)' : 'Coffee Cup', 
':-{}' : 'Cold', 
'' : 'Cold', 
'?_?' : 'Cold Stare', 
'$:-)' : 'Comb Stuck in the Hair', 
'(;)(-_-)' : 'Comforting', 
'(co)' : 'Computer', 
'_____' : '', 
'|O-O|' : '', 
'|:::|' : '', 
'-----' : 'Computer', 
'@/:^)' : 'Conan', 
':-r' : 'Concentrating', 
':-8(' : 'Condescending', 
':^)' : 'Confounded', 
'O.o' : 'Confused', 
':-Q' : 'Confused', 
':-/' : 'Confused', 
':-S' : 'Confused', 
':s' : 'Confused', 
':-$' : 'Confused', 
'(@_@)' : 'Confused', 
'-' : 'Confused', 
'=?' : 'Confused', 
",':(" : "Confused",
'O.o?' : 'Confused', 
':-??' : 'Confused', 
'0' : '0', 
'__' : 'Confused', 
'(p_q)' : 'Confused', 
'\(_o)/' : 'Confused', 
'?:|' : 'Confused', 
'-.-?' : 'Confused', 
'(-?)' : 'Confused', 
'()' : 'Confused', 
'(-_-)' : 'Confused', 
':?' : 'Confused', 
':-?' : 'Confused', 
':???:' : 'Confused', 
'o_O' : 'Confused', 
'(??)' : 'Confused', 
'8-/' : 'Confused', 
'8/' : 'Confused', 
'' : 'Content', 
'B-)' : 'Cool', 
'0' : '0', 
'-' : 'Cool', 
'qB-)' : 'Cool', 
'-D' : 'Cool', 
'8=)' : 'Cool', 
'B=)' : 'Cool', 
'(cool)' : 'Cool', 
':B)' : 'Cool', 
'B)' : 'Cool', 
'-<0oo' : 'Corn', 
'3:-O' : 'Cow', 
'3:O' : 'Cow', 
'3:08' : 'Cow', 
'<):-)' : 'Cowboy', 
'}:-)' : 'Cowboy', 
'<):)' : 'Cowboy', 
'C):)' : 'Cowboy', 
'(-:(3' : 'Cowboy', 
'C):-)' : 'Cowboy', 
'{:-)' : 'Cowgirl', 
')O{)|><' : 'Cowgirl', 
'V.v.V' : 'Crab', 
'V.^.' : 'V', 
'(\/)!_!(\/)' : 'Crab', 
'' : 'Crab', 
'()_RED_(>' : 'Crayon', 
'())Crayola))>' : 'Crayon', 
'(_)' : 'Crazed', 
'%-)' : 'Crazy', 
'()' : 'Crazy', 
'>,=,-' : 'Crocodile', 
':_(' : 'Crying', 
':*(' : 'Crying', 
'T_T' : 'Crying', 
':((' : 'Crying', 
":'(" : "Crying",
';_;' : 'Crying', 
'QQ' : 'Crying', 
':..[' : 'Crying', 
'T^T' : 'Crying', 
"='(" : "Crying",
'8:::(' : 'Crying', 
'TT^TT' : 'Crying', 
'TT_TT' : 'Crying', 
'i_i' : 'Crying', 
":'-C" : "Crying",
';-;' : 'Crying', 
':"-(' : 'Crying', 
':~-(' : 'Crying', 
'TT.TT' : 'Crying', 
'(!_!)' : 'Crying', 
'&.(..' : 'Crying', 
'(ToT)' : 'Crying', 
'(;_;)' : 'Crying', 
":'-(" : "Crying",
';(' : 'Crying', 
'Y.Y' : 'Crying', 
'T.T' : 'Crying', 
':cry:' : 'Crying', 
';=(' : 'Crying', 
'(cry)' : 'Crying', 
'(-__-)' : 'Crying', 
'|_|' : 'Cup', 
'C(_)' : 'Cup', 
'|_P' : 'Cup', 
'C(_)' : 'Cup of Coffee', 
'|_P' : 'Cup of Coffee', 
'~o)' : 'Cup of Coffee', 
'[_]P' : 'Cup of Coffee', 
'@:-)' : 'Curly Hair', 
'8:-)' : 'Curly Hair', 
':-@!' : 'Cursing', 
'X3' : 'Cute', 
'n_n' : 'Cute', 
'||' : 'Cutting a rug', 
'@-|' : 'Cyborg', 
'()' : 'Cyborg', 
'o-)' : 'Cyclops', 
'O-(' : 'Cyclops', 
'.)' : 'Cyclops', 
'Q-)' : 'Cyclops', 
'*-(' : 'Cyclops', 
'>P-(' : 'Cyclops', 
':/7)' : 'Cyrano de Bergerac', 
'Emoticons' : 'that start with D', 
'Emoticon' : 'Meaning', 
':)-S=' : 'Dancer', 
'\:D/' : 'Dancing', 
'<(..<)' : 'Dancing', 
'(>..)>' : 'Dancing', 
'~(o.o)~' : 'Dancing', 
'(^_^)(^_^)' : 'Dancing', 
'v(*.*)^' : 'Dancing', 
'(~^.^)~' : 'Dancing', 
'(dance)' : 'Dancing', 
'\o/' : 'Dancing', 
'(.)' : 'Dancing', 
'^^)' : 'Dancing', 
'(o)' : 'Dancing', 
'(o)' : 'Dancing', 
'C8<]' : 'Darth Vader', 
'()' : 'Darth Vader', 
'Q:|' : 'Davey Crockett', 
'8->' : 'Daydreaming', 
'*-*' : 'Dazed', 
'**' : 'Dazed', 
'[*_*]' : 'Dazzled', 
'x_x' : 'Dead', 
'-' : 'Dead', 
'x.x' : 'Dead', 
'XP' : 'Dead', 
'8-#' : 'Dead', 
'X' : 'X', 
'U' : 'Dead', 
'(X_X)' : 'Dead', 
'(+_+)' : 'Dead', 
"'-=,o" : "Dead",
'(#_)' : 'Dead', 
'>T_T' : 'Dead', 
'(u_u)' : 'Dead', 
'(==)' : 'Dead cat', 
'()' : 'Deceased', 
'~(O)|' : 'Deep Sea Diver', 
'3:*)' : 'Deer', 
'(T_T)->' : 'Defeated', 
'(_)' : 'Defeated', 
'^(#)_' : 'Demon', 
'(<_>)' : 'Depressed', 
'._.' : 'Depressed', 
'-----' : 'Depressed', 
'(._.)' : 'Depressed', 
'c_*' : 'Depressed', 
'_' : 'Depressed', 
'(;)' : 'Depressed', 
':-[' : 'Despondent', 
'i-)' : 'Detective', 
'()' : 'Devastated', 
'>:)' : 'Devil', 
'(6)' : 'Devil', 
']:->' : 'Devil', 
'}:-)' : 'Devil', 
'()}:o)' : 'Devil', 
'}=^{|~' : 'Devil', 
'3:)' : 'Devil', 
'>:-)~' : 'Devil', 
'>;->' : 'Devil', 
'3:-)' : 'Devil', 
':evil:' : 'Devil', 
'(devil)' : 'Devil', 
':[' : 'Devil', 
':-[' : 'Devil', 
'>:->' : 'Diabolical', 
'[](*' : 'Dialing', 
'{>O' : 'Diamond Ring', 
":-')" : "Dimple",
'/\"' : 'Dinosaur', 
'_..._' : '', 
'=|:-)X' : 'Diplomat', 
':-|' : 'Disappointed', 
':|' : 'Disappointed', 
':(&' : 'Disappointed', 
'=\"' : 'Disappointed', 
'Disappointe' : 'd', 
'_' : 'Disapproval', 
'/' : 'Discarding', 
'' : 'Discontent', 
'DX' : 'Disgusted', 
':' : 'Disgusted', 
'-__-' : 'Disinterested', 
'(*m)' : 'Dissatisfied', 
'/:(' : 'Distraught', 
'_' : 'Distressed', 
'%-}' : 'Dizzy', 
'd[-_-]b' : 'DJ', 
'[:-}' : 'DJ', 
'd^_^b' : 'DJ', 
':-$' : 'Do Not Tell Anyone', 
':-)-o' : 'Doctor', 
'O:v)>--o' : 'Doctor', 
':O?' : 'Dog', 
':O/' : 'Dog', 
':O$' : 'Dog', 
':O)' : 'Dog', 
'<[.,=.,]-' : 'Dog', 
'<[~.~]>' : 'Dog', 
':o3' : 'Dog', 
'<*^)' : '', 
'<*^)' : '', 
':3-<' : 'Dog', 
'/(^.^)\"' : 'Dog', 
'(UxU)' : 'Dog', 
'UU' : 'Dog', 
'' : 'Dog', 
'U)~' : 'Dog', 
'' : 'Dog', 
'' : 'Dog', 
'8==3' : 'Dog Bone', 
'&===3' : 'Dog bone', 
'(&)' : 'Dog Face', 
'VV' : 'Doggy', 
'(>.<)' : 'Doh', 
'[$(1)$]' : 'Dollar', 
'===:-D' : 'Don King', 
'(8)<=' : 'Doraemon', 
':D))' : 'Double Chin', 
':' : 'T Doubtful', 
'}:-(=' : 'Dracula', 
'(>:^F)' : 'Dracula', 
'^.=.^' : 'Dragon', 
'<*,^^,--' : 'Dragon', 
'I-)' : 'Dreaming', 
'*DRINK*' : 'Drink', 
'\%/' : 'Drink', 
'=P~' : 'Drooling', 
':-P' : 'Drooling', 
':P' : 'Drooling', 
'*DROOLING*' : 'Drooling', 
':)_' : 'Drooling', 
':-B' : 'Drooling', 
'(**)' : 'Drooling', 
'~~~~\o/~~~~' : 'Drowning', 
':*)' : 'Drunk', 
':*)?' : 'Drunk', 
'%-<I>' : 'Drunk', 
':#)' : 'Drunk', 
'(drunk)' : 'Drunk', 
'o_O' : 'Drunk', 
'_|V|_' : 'Duck', 
'8()' : 'Duck', 
'\_O<' : 'Duck', 
':-F' : 'Dufus', 
'|-(' : 'Dull', 
'|(' : 'Dull', 
'|=(' : 'Dull', 
'p^b' : 'Dumb', 
'8)' : 'Dumb Person', 
'Oo!' : 'Dumbfounded', 
'<:-|' : 'Dunce', 
'<:-)' : 'Dunce', 
'()))TNT)))-x' : 'Dynamite', 
'Emoticons' : 'that start with E', 
'Emoticon' : 'Meaning', 
'' : 'Eager', 
'p^_^q' : 'Earphones', 
'::--((' : 'Earthquake', 
':g' : 'Eating', 
':K' : 'Eating', 
'()' : 'Edward Scissorhands', 
'()' : 'Edward Scissorhands', 
'(:-|' : 'Egghead', 
'~~~\8-O' : 'Electrocuted', 
'\/\/\/\,8-O' : 'Electrocuted', 
':~' : 'Elephant', 
'~:3' : 'Elephant', 
'\_/' : 'Elephant', 
"~MC'~" : "Elephant",
'}.' : 'Elephant', 
'*<:-)-:--:' : 'Elf', 
'' : 'Elton John', 
'@=)' : 'Elvis', 
'2:)' : 'Elvis', 
'?:-p' : 'Elvis', 
':-[]' : 'Elvis', 
'5:-)' : 'Elvis Presley', 
'&:-)' : 'Elvis Presley', 
':-$' : 'Embarrassed', 
':$' : 'Embarrassed', 
':-[' : 'Embarrassed', 
'^^"' : 'Embarrassed', 
'(#^.^#)' : 'Embarrassed', 
':oops:' : 'Embarrassed', 
'(*/*)' : 'Embarrassed', 
'Embarrasse' : 'd', 
'(##)' : 'Embarrassed', 
'//_^' : 'Emo', 
'T-T' : 'Emo', 
'(//.^)' : 'Emo', 
'(//_^)' : 'Emo', 
'(//_-)' : 'Emo', 
'c//_+.' : 'Emo', 
"c//_^'" : "Emo",
'c/_-*' : 'Emo', 
'(-_\\\)' : 'Emo', 
'//_T)o.' : 'Emo', 
'd(//^)b' : 'Emo', 
'\/O\/O\/' : 'Emo', 
'(emo)' : 'Emo', 
',_' : 'Emo', 
'(///.-)' : 'Emo', 
'' : 'Enamored', 
'(*m*)' : 'Enraged', 
'(v)' : 'Envious', 
'(envy)' : 'Envious', 
'(@){' : 'Euphonium', 
'((((((^_^;)' : 'Evading', 
'>:3' : 'Evil Cuteness', 
'(.\|/.)' : 'Evil Face', 
'):D}' : 'Evil Face', 
':{)' : 'Evil Face', 
'>:~>' : 'Evil Face', 
'>:{)>' : 'Evil Face', 
']}:-)>' : 'Evil Face', 
'>-)' : 'Evil Grin', 
'>:)' : 'Evil Grin', 
']:)' : 'Evil Grin', 
':twisted:' : 'Evil Grin', 
'(grin)' : 'Evil Grin', 
'>:D' : 'Evil Laugh', 
'()' : 'Evil Laugh', 
'>=)' : 'Evil Smile', 
'/^^\!' : 'Exasperated', 
':-O' : 'Excited', 
'(*-*)' : 'Excited', 
':o' : 'Excited', 
'(^O^)' : 'Excited', 
'()' : 'Excited', 
'#-)' : 'Exhausted', 
'||' : 'Exhausted', 
'(;?`)' : 'Exhausted', 
'STO' : 'Exhausted', 
'(((ToT)~~' : 'Exorcism', 
'_(*)' : 'Exorcism', 
'<>' : 'Eye', 
'P-)' : 'Eye Patch', 
'b-)' : 'Eye Patch', 
'' : 'Eye Roll With Hands Up', 
'}=D' : 'Eyebrows', 
'-@--@-' : 'Eyeglasses', 
'Emoticons' : 'that start with F', 
'Emoticon' : 'Meaning', 
':&' : 'Fail', 
'(6.6)' : 'Fainting', 
'(_)' : 'Fake Smile', 
'?:-)' : 'Fancy Hair', 
'o,..,o' : 'Fangs', 
'^,..,^' : 'Fangs', 
'(0)' : 'Fed Up', 
'=^O.O^=' : 'Felix the Cat', 
'' : 'O', 
'+' : 'Female', 
'' : 'O', 
'' : '', 
'^' : 'Female', 
'O<-<=' : 'Female', 
';~[' : 'Fighter', 
'' : '', 
'(~)' : 'Filmstrip', 
'(yn)' : 'Fingers Crossed', 
'X=' : 'Fingers Crossed', 
'()))))-x' : 'Firecracker', 
'}-(((*>' : 'Fish', 
'<")))><' : 'Fish', 
'><(((">' : 'Fish', 
'<.{{{><' : 'Fish', 
'{}{' : 'Fish', 
'<><' : 'Fish', 
'<((((><' : 'Fish', 
"<('O')>" : "Fish",
'<*)))<' : 'Fish', 
'|>///^<' : 'Fish', 
'>)##)' : 'Fish', 
')' : 'Fish', 
'()#))<<' : 'Fish', 
"<')+++<" : "Fish Bone",
'(~)' : 'Fish Bowl', 
'p#d' : 'Fist Bump', 
'i-=<***' : 'Flamethrower', 
"!.'v" : "Flat Top",
'OZ~<' : 'Flexing', 
'(_)' : 'Flexing', 
'()' : 'Flexing', 
'()' : 'Flipping Table', 
'()' : 'Flipping Table', 
'()' : 'Flipping Table', 
';D' : 'Flirting', 
';-)' : 'Flirty', 
'(ff)' : 'Flirty Female', 
';3' : 'Flirty Grin', 
'(fm)' : 'Flirty Male', 
'@>---' : 'Flower', 
'@>--;--' : 'Flower', 
'<-#' : 'Flower', 
"@-'-" : "Flower",
'(f)' : 'Flower', 
'(flower)' : 'Flower', 
'###' : 'Flower', 
'0==I==,==I===' : 'Flute', 
"D';" : "FML",
'O-&-<' : 'Folding Arms', 
':-!' : 'Foot in Mouth', 
'()' : 'Football', 
'G:-)' : 'Football Helmet', 
'=:-H' : 'Football Player', 
'C:#' : 'Football Player', 
'---E' : 'Fork', 
'o^o-o|_' : 'Forklift', 
'|:-<' : 'Forlorn', 
'%%-' : 'Four Leaf Clover', 
'88-' : 'Four Leaf Clover', 
'Vm~' : 'Fox', 
':^{=' : 'Frank Zappa', 
'E:/' : 'Frankenstein', 
'[:-|I' : 'Frankenstein', 
'[8-]' : 'Frankenstein', 
'()' : 'Frantic', 
'()' : 'Frantic', 
'()' : 'Freak Out', 
'()' : 'Freak Out', 
":'|" : "Freezing",
':pd:' : 'French Kiss', 
':bq:' : 'French Kiss', 
':bd:' : 'French Kiss', 
'(`*)' : 'Friendly', 
'O' : 'O', 
'8-|' : 'Frightened', 
'()' : 'Frightened', 
'8)~~*' : 'Frog', 
'8)' : 'Frog', 
':(?)' : 'Frog', 
':()' : 'Frog', 
'[II]D' : 'Frosty Mug', 
'' : 'Frosty the Snowman', 
'' : 'Frosty the Snowman', 
':-(' : 'Frown', 
':(' : 'Frown', 
':o{' : 'Frown', 
':-(' : 'Frowning', 
'=P' : 'Frustrated', 
'X[' : 'Frustrated', 
':-P' : 'Frustrated', 
'><' : 'Frustrated', 
'B/' : 'Frustrated', 
'>.<*' : 'Frustrated', 
'-_-*' : 'Frustrated', 
':-L' : 'Frustrated', 
'()' : 'Fu Manchu Man', 
'(fubar)' : 'FUBAR', 
'|-[#]' : 'Fuming', 
'B-}' : 'Funny', 
'(_)' : 'Furious', 
'*:*' : 'Fuzzy', 
'Emoticons' : 'that start with G', 
'Emoticon' : 'Meaning', 
':-(*)' : 'Gag Me', 
'qB]' : 'Gangster', 
'@}:-)' : 'Gardener', 
'=:o' : 'Gasp', 
'' : 'Geek', 
'3>(' : 'Geezer', 
'(D:-]' : 'General', 
'@:i' : 'Genie', 
'{@:)}' : 'Genie', 
'4:-)' : 'George Washington', 
'd=P' : 'Ghetto', 
'(>")>' : 'Ghost', 
'<(")>' : 'Ghost', 
'(",)' : 'Ghost', 
'(boo)' : 'Ghost', 
'()' : 'Ghost', 
'((()))' : 'Ghost', 
'8[+]' : 'Gift', 
'()' : 'Gift', 
'(g)' : 'Gift with a Bow', 
';))' : 'Giggle', 
'*GIGGLING*' : 'Giggling', 
'(giggle)' : 'Giggling', 
'(chuckle)' : 'Giggling', 
'()' : 'Giggling', 
'>:o===Q<' : 'Giraffe', 
'(x)' : 'Girl', 
'8:^)' : 'Girl', 
'(-_-)#' : 'Girl', 
'~0(**)0~' : 'Girl', 
'(;`)' : 'Giving Up', 
'' : 'Glad', 
'>:[' : 'Glaring', 
'\~/' : 'Glass', 
'\_/' : 'Glass', 
'??' : 'Glasses', 
'-o-o-' : 'Glasses', 
'8-)' : 'Glasses', 
'()-()' : 'Glasses', 
'-@--@-' : 'Glasses', 
'\o-o/' : 'Glasses', 
'(-_q)' : 'Glasses', 
'8)' : 'Glasses', 
'B-)' : 'Glasses', 
'B)' : 'Glasses', 
'(^0_0^)' : 'Glasses', 
'' : 'Gleeful', 
'(nah)' : 'Goat', 
'(nnh)' : 'Goat', 
':-{>' : 'Goatee', 
';' : 'Goatee', 
'):()-' : 'Goatee', 
'o/' : 'Golf', 
'()' : 'Golf Clap', 
'' : 'Good Morning', 
'(`)' : 'Good Night', 
'(ToT)/~~~' : 'Goodbye', 
':8]' : 'Gorilla', 
'%:O' : 'Gorilla', 
'8:]' : 'Gorilla', 
'|<:-)' : 'Graduate', 
'L:-)' : 'Graduate', 
'Q:-)' : 'Graduate', 
'oO:)&' : 'Grandma', 
'' : 'Grandma', 
'(~};)' : 'Grateful Dead', 
'$_$' : 'Greedy', 
'($v$)' : 'Greedy', 
'$__$' : 'Greedy', 
".'!" : "Grim",
':->' : 'Grin', 
':D' : 'Grin', 
'-D' : 'Grin', 
':oP' : 'Grin', 
'' : 'Grinning', 
'(-_' : 'Groggy', 
'v(*.*)^' : 'Grooving', 
'(^o^)' : 'Grooving', 
':-!' : 'Gross', 
"l:['" : "Groucho Marx",
'[>#<]' : 'Grouchy', 
'(.(.).)' : 'Group', 
'>:(' : 'Grumpy', 
'^i^' : 'Guardian Angel', 
'>I<' : 'Guardian Angel', 
'^_^;' : 'Guilty', 
'(____)' : 'Guitar', 
'(_|||_)' : 'Guitar', 
'=' : 'Gun', 
'' : 'Gun', 
'' : 'Gun', 
'' : 'Gun', 
'S:~)' : 'Guy', 
'$8()' : 'Guy with a Toupee', 
'Emoticons' : 'that start with H', 
'Emoticon' : 'Meaning', 
'*8D' : 'Hair', 
'((:D' : 'Hair', 
'~~8-O' : 'Hair', 
'@:-)' : 'Hair', 
'#:-)' : 'Hair', 
'~#:-(' : 'Hair', 
'?:-)' : 'Hair', 
'%-.-%' : 'Hair', 
"@.'v" : "Hair",
'{:-{)}' : 'Hairy', 
'L-:' : 'Half Smile', 
'(||]' : 'Hamburger', 
'(%)' : 'Handcuffs', 
'(:>-<' : 'Hands Up', 
':-|><' : 'Hands Up', 
'(handshake)' : 'Handshake', 
'&:-]' : 'Handsome', 
'=)' : 'Happy', 
'U' : 'Happy', 
':-)' : 'Happy', 
':)' : 'Happy', 
'^.^' : 'Happy', 
'^^' : 'Happy', 
'(:' : 'Happy', 
'|___|' : 'Happy', 
'^-^' : 'Happy', 
'8D' : 'Happy', 
'(^_^)' : 'Happy', 
'8^)' : 'Happy', 
':-}' : 'Happy', 
'!_!' : 'Happy', 
':)))' : 'Happy', 
'(happy)' : 'Happy', 
'()' : 'Happy', 
'' : 'Happy', 
'' : 'Happy', 
'o' : 'Happy', 
'B^)' : 'Happy', 
'8-D' : 'Happy', 
'BD' : 'Happy', 
'B^D' : 'Happy', 
'8^D' : 'Happy', 
'' : 'Happy', 
'*<|:o)' : 'Happy Birthday', 
'()' : 'Happy crying', 
':")' : 'Happy Tears', 
'|||)' : 'Harp', 
'<~8)' : 'Harry Potter', 
'>O<' : 'Harry Potter Snitch', 
'#' : 'Hashtag', 
'q=)' : 'Hat', 
'xd:)' : 'Hat', 
'*<:)' : 'Hat', 
'+(:-)' : 'Hat', 
"*<.'v" : "Hat",
"d.'v" : "Hat",
'\,,/(>.<)\,,/' : 'Head Banging', 
'(({..}))' : 'Headache', 
'(|:-)' : 'Headband', 
'>:^(' : 'Headhunter', 
'oo' : 'Headlights', 
'0(o.o)0' : 'Headphones', 
'd(-_-)b' : 'Headphones', 
'[:-)' : 'Headphones', 
"q[-'.'-]p" : "Headphones",
'dL-_-b' : 'Headset', 
'<3' : 'Heart', 
'<u3' : 'Heart', 
'S2' : 'Heart', 
'' : 'Heart', 
'-(`.)->' : 'Heart', 
'(h)' : 'Heart', 
'(l)' : 'Heart', 
'(heart)' : 'Heart', 
'./\..|..|....' : 'Heart Monitor', 
'()' : 'ol Heartbroken', 
'\,,/' : 'Heavy Metal', 
'(heidy)' : 'Heidy', 
'o_o' : 'Hello', 
'(o)' : 'Help', 
'()!!' : 'Help', 
'?:)#' : 'Hema Employee', 
'' : 'Herp Derp', 
'(_;)_)' : 'Hiding', 
'(_|' : 'Hiding', 
'|)' : 'Hiding', 
'|_)' : 'Hiding', 
'~~' : 'High Brow', 
'(h5)' : 'High Five', 
'o/\o' : 'High Five', 
'^_^/' : 'High Five', 
'(*)(*)' : 'High Five', 
'^y^-/|\--Y^' : 'Highway', 
'o->' : 'Hiro', 
':-/' : 'Hmmm', 
'(#]' : 'Hockey Player', 
'L.' : 'Hockey Stick', 
'L.' : 'Hockey Stick and Puck', 
'8-)' : 'Hollywood', 
'(_8^(|)' : 'Homer', 
'(8-(1)' : 'Homer', 
'(_8^(1)' : 'Homer Simpson', 
'~(_8^(|)' : 'Homer Simpson', 
'o--}' : 'Hope', 
'o(^^o)(o^^)o' : 'Hopeful', 
'}:)' : 'Horny', 
'O_' : '0', 
'[_]' : 'Horror', 
']:-:)' : 'Horse', 
'(h)' : 'Hot', 
'(;)' : 'How Should I Know', 
'(:)\/<' : 'Hug', 
':)X' : 'Hug', 
"(>'')><(''<)" : "Hug",
'(((H)))' : 'Hug', 
'{}' : 'Hug', 
'({})' : 'Hug', 
'(bear)' : 'Hug', 
'(hug)' : 'Hug', 
'()' : 'Hug', 
'{*}' : 'Hug and Kiss', 
'd=()=b' : 'Hug me', 
'<(^.^<)' : 'Huge', 
'(>^_^)><(^o^<)' : 'Hugging', 
'<(*.*<)' : 'Huggle', 
':D<' : 'Hugs', 
'((()))' : 'Hugs', 
'{{{***}}}' : 'Hugs and Kisses', 
'[.]' : 'Huh', 
'~!~' : 'Humbled', 
'%*@:-(' : 'Hung Over', 
':0' : 'Hungry', 
':-[]' : 'Hungry', 
':!!' : 'Hurry Up', 
'XD' : 'Hyper', 
'U' : 'Hypnotist', 
'@-)' : 'Hypnotized', 
'Emoticons' : 'that start with I', 
'Emoticon' : 'Meaning', 
':^)' : 'I Do Not Know', 
'X_X' : 'I Do Not Want to See', 
'_\m/' : 'I Love You', 
'~oO>' : 'Ice Cream Cone', 
'<OO' : 'Ice Cream Cone', 
'<Oo*' : 'Ice Cream Cone', 
'<><' : 'Ichthus', 
'*-:)' : 'Idea', 
':idea:' : 'Idea', 
'*help' : 'IM', 
'*red+u' : 'IM', 
':!:' : 'Important', 
'O-S-<' : 'In a hurry', 
'(@)' : 'In a Nut Shell', 
'(*_*)' : 'In Love', 
'<3+<3' : 'In Love', 
'(_)' : 'In Love', 
':]' : 'In Love', 
'(inlove)' : 'In Love', 
'(love)' : 'In Love', 
':-]' : 'In Love', 
'._.' : 'Inadequate', 
'_/\,' : 'Inch Worm', 
':-S' : 'Incoherent', 
'(o_O)' : 'Incredulous', 
':7/' : 'Indecisive', 
':-|' : 'Indifferent', 
':-I' : 'Indifferent', 
'-' : 'Indifferent', 
'E_E' : 'Indifferent', 
':neutral:' : 'Indifferent', 
'**' : 'Infatuated', 
'(*)=3' : 'Infatuated', 
'()' : 'Infatuated', 
'0:-)' : 'Innocent', 
'O:D' : 'Innocent', 
'(wm)' : 'Innocent', 
')' : 'Insane', 
'?_?' : 'Irritated', 
'%7<' : 'Irritated', 
'(ip)' : 'Island with a Palm Tree', 
'^#(^' : 'It Was Not Me', 
'(wasntme)' : 'It Was Not Me', 
':<|' : 'Ivy League', 
'Emoticons' : 'that start with J', 
'Emoticon' : 'Meaning', 
'(:-|K-' : 'James Bond', 
'.().' : 'Jamming to music', 
'8(>_<)8' : 'Jealous', 
'8-)' : 'Joe Cool', 
'//o-o\\' : 'John Lennon', 
'((*J*))' : 'John Lennon', 
'.-' : 'John Lennon', 
'X-p' : 'Joking', 
'(jk)' : 'Joking', 
'*JOKINGLY*' : 'Jokingly', 
':-]' : 'Jovial', 
'\m/<(^_^)>\m/' : 'Joyful', 
'()/' : 'Joyful', 
'o(^o^)o' : 'Joyful', 
'()' : 'Joyful', 
'Emoticons' : 'that start with K', 
'Emoticon' : 'Meaning', 
'(:-#' : 'Keeping Quiet', 
'c",)' : 'Kid', 
'K:P' : 'Kid', 
'X:-)' : 'Kid', 
'\VVV/' : 'King', 
'\$$$/' : 'King', 
']:)|--<' : 'King', 
'<(^.^)>' : 'Kirby', 
"<('.'<)" : "Kirby",
"(>'.')>" : "Kirby",
'<(^_^)>' : 'Kirby', 
"<('.')>" : "Kirby",
"<('-'<)" : "Kirby",
"(>'')>" : "Kirby",
'c(-_-c)' : 'Kirby', 
'<(^,..,^)>' : 'Kirby', 
'(^^)' : 'Kirby', 
'*' : 'Kiss', 
':-)*' : 'Kiss', 
':-X' : 'Kiss', 
':-*' : 'Kiss', 
';-(!)' : 'Kiss', 
':*)' : 'Kiss', 
'8^*' : 'Kiss', 
':+' : 'Kiss', 
':P' : 'Kiss', 
':*' : 'Kiss', 
'X(*/*)x' : 'Kiss', 
'(3)' : 'Kiss', 
'(xo)' : 'Kiss', 
':-{}' : 'Kiss', 
'(^3^)' : 'Kiss', 
'(*^3^)/~' : 'Kiss', 
'(*^^)^*)' : 'Kiss', 
':=*' : 'Kiss', 
'(kiss)' : 'Kiss', 
'()`)' : 'Kiss', 
'(_X_)' : 'Kiss my Butt', 
'' : 'Kiss my butt', 
'*KISSED*' : 'Kissed', 
':}{:' : 'Kisses', 
'*KISSING*' : 'Kissing', 
'(}{)' : 'Kissing', 
'(-}{-)' : 'Kissing', 
':**:' : 'Kissing', 
'(=^-^=)' : 'Kitten', 
'=^.^=' : 'Kitty', 
'=+=' : 'Kitty', 
':3' : 'Kitty', 
'=^w^=' : 'Kitty', 
'(=^--^=)' : 'Kitty', 
'(=^^=)' : 'Kitty', 
'[|]' : 'Kleenex', 
'>>:-|' : 'Klingon', 
'.-=o=-.' : 'Klingon Bird of Prey', 
'+<#^v' : 'Knight', 
'+<||-)' : 'Knight', 
'@(*o*)@' : 'Koala', 
'' : 'Koala', 
'' : 'Koala', 
'' : 'Koala', 
':*)O' : 'Kool', 
'' : 'Kyubey', 
'Emoticons' : 'that start with L', 
'Emoticon' : 'Meaning', 
'&0(80<' : 'Lady', 
'<{::}>' : 'Ladybug', 
'(::)<' : 'Ladybug', 
'-.-' : 'Lame', 
'' : 'Larry David', 
'l8r)' : 'Later', 
':))' : 'Laughing', 
'XD' : 'Laughing', 
':L' : 'Laughing', 
":'D" : "Laughing",
'=^D' : 'Laughing', 
'>8-D' : 'Laughing', 
'(^_^)' : 'Laughing', 
'()' : 'Laughing', 
'-*)' : 'Laughing', 
':o' : 'Laughing', 
'' : 'Laughing', 
'' : 'Laughing', 
'=-D' : 'Laughing', 
'=D' : 'Laughing Out Loud', 
'X-D' : 'Laughing Out Loud', 
'(lol)' : 'Laughing Out Loud', 
':lol:' : 'Laughing Out Loud', 
':.-)' : 'Laughing Tears', 
'({)' : 'Left Hug', 
'()' : 'Lethargic', 
':^o' : 'Liar Liar', 
':PdPdP' : 'Licking', 
':-9' : 'Licking', 
'(`)' : 'Licking', 
'#-)' : 'Lieutenant LaForge', 
'(i)' : 'Light Bulb', 
'(li)' : 'Lightning', 
'(Y)' : 'Like', 
':3' : 'Lion', 
':-*' : 'Lips are Sealed', 
':-#' : 'Lips are Sealed', 
':-X' : 'Lips are Sealed', 
':X' : 'Lips are Sealed', 
':#' : 'Lips are Sealed', 
':=x' : 'Lips are Sealed', 
':=#' : 'Lips are Sealed', 
':()' : 'Lipstick', 
':-{}' : 'Lipstick', 
'd-_-b' : 'Listening to Music', 
'd^_^b' : 'Listening to Music', 
'd(^_^)b' : 'Listening to Music', 
'o(^_^)o' : 'Listening to music', 
'd(-\\\)b' : 'Listening to music', 
'_\\//' : 'Live Long and Prosper', 
'>-<' : 'Livid', 
'(.`.)' : 'Livid', 
'(**)' : 'Livid', 
'~~I==I>' : 'Lizard', 
'' : 'Lobster', 
'@---' : 'Lollipop', 
':P@---' : 'Lollipop', 
':~)' : 'Long Nose', 
'(*)?(*)' : 'Looking', 
'>_>' : 'Looking', 
'<_<' : 'Looking', 
'>.>' : 'Looking', 
'\\(<.<|>.>)//' : 'Looking', 
'(_)' : 'Looking', 
'<o?o>' : 'Looking at You', 
'<*_*>' : 'Looking at You', 
'L-)' : 'Loser', 
':-{}' : 'Lots of Lipstick', 
'<3' : 'Love', 
'~~' : 'Love', 
':x' : 'Love Struck', 
'__.' : 'Lying Down', 
'o@=<' : 'Lying Down', 
'Emoticons' : 'that start with M', 
'Emoticon' : 'Meaning', 
'=/' : 'Mad', 
'>=(' : 'Mad', 
'>,<' : 'Mad', 
'~:(' : 'Mad', 
'>=[' : 'Mad', 
'>}:-(' : 'Mad', 
':U' : 'Mad', 
')8^(' : 'Mad', 
':!' : 'Mad', 
">;-('" : "Mad",
'>_<' : 'Mad', 
':x' : 'Mad', 
':-x' : 'Mad', 
':mad:' : 'Mad', 
'(#)' : 'Mad', 
':-.)' : 'Madonna', 
'{8-*' : 'Maggie Simpson', 
'*)' : 'Magic Casting', 
'<*:-)' : 'Magician', 
'(malthe)' : 'Malthe', 
'0-|-[' : 'Man', 
'@@@@:-)' : 'Marge Simpson', 
'(@@@@):^)' : 'Marge Simpson', 
'######:o)' : 'Marge Simpson', 
'@@@:)' : 'Marge Simpson', 
'(#-[' : 'Marine', 
':-)(-:' : 'Married', 
'' : '', 
'' : 'Married', 
'-----(_)-' : 'Marshmallow', 
'-()_)()_)' : 'Marshmallow', 
'(.!.)' : 'Martian', 
'=:)' : 'Martian', 
'Q>--|' : 'Martini', 
'(d)' : 'Martini Glass', 
'|>--|' : 'Martini Glass', 
'(drink)' : 'Martini Glass', 
'>:-I' : 'Mean', 
'@|-)' : 'Meditating', 
'(-_-)' : 'Meditating', 
'' : 'Meh', 
'' : 'Mellow', 
'O' : 'O', 
'@' : 'Mention', 
'@_@' : 'Mesmerized', 
'\m/' : 'Metal', 
'lml' : 'Metal', 
'lml_' : 'Metal', 
'|:-[|]' : 'Mick Jagger', 
'8(:-)' : 'Mickey Mouse', 
'O' : 'Mickey Mouse', 
'o/<' : 'Midget', 
'(o:v)' : 'Miner', 
':p' : 'Mischievous', 
':}' : 'Mischievous', 
'?8}' : 'Mischievous', 
'=)=' : 'Mixed Feelings', 
':D:' : 'Mixed Feelings', 
'(mm)' : 'Mmm', 
'(mp)' : 'Mobile Phone', 
'^o^' : 'Mog', 
'-:-)' : 'Mohawk', 
"=.'v" : "Mohawk",
'(mo)' : 'Money', 
'[*(1)*]' : 'Money', 
'$-)' : 'Money Eyes', 
':-$' : 'Money Mouth', 
':-($)' : 'Money Mouth', 
'O|-)' : 'Monk', 
'+<:-|' : 'Monk', 
'@(-_-)@' : 'Monkey', 
':(|)' : 'Monkey', 
'U' : 'Monkey', 
"@('.')@" : "Monkey",
'---' : 'Monkey', 
"@'o'@" : "Monkey",
'@(^_^)@' : 'Monkey', 
'@(*_*)@' : 'Monkey', 
'()=' : 'Monkey', 
'(;_)' : 'Monocle', 
'_' : 'Monocle', 
':E' : 'Monster', 
"';..;'" : "Monster",
'VVV' : 'Monster', 
'>:(><)' : 'Monster', 
'(_i_)' : 'Mooning', 
'(mooning)' : 'Mooning', 
'(|)' : 'Mooning You', 
'(>|<)' : 'Mosquito', 
'o-7-o' : 'Motorbike', 
'o--7o' : 'Motorcycle', 
':o&o>' : 'Motorcycle', 
'^v^v^' : 'Mountain', 
'~<>^>' : 'Mouse', 
'oO~' : 'Mouse', 
'<:3)~' : 'Mouse', 
'<:3)~~' : 'Mouse', 
'<:3)~~~' : 'Mouse', 
'8(:-)' : 'Mouseketeer', 
'(~)' : 'Movie', 
'(film)' : 'Movie', 
'(movie)' : 'Movie', 
'(m)' : 'MSN Messenger', 
'cl_l' : 'Mug', 
'[II]D' : 'Mug', 
'(_)' : 'Mug', 
'(flex)' : 'Muscle', 
'(muscle)' : 'Muscle', 
'' : 'Muscular Person', 
'(music)' : 'Music', 
':-{' : 'Mustache', 
':{' : 'Mustache', 
'={D' : 'Mustache', 
':{D' : 'Mustache', 
':{)' : 'Mustache', 
':-#|' : 'Mustache', 
':-3' : 'Mustache', 
':{|' : 'Mustache Man', 
':-:' : 'Mutant', 
'(myspace)' : 'Myspace', 
'Emoticons' : 'that start with N', 
'Emoticon' : 'Meaning', 
':_:' : 'Nagato', 
':-SS' : 'Nailbiting', 
'(Z(:^P' : 'Napoleon', 
'~,~' : 'Napping', 
'_:^)' : 'Native American', 
':3' : 'Naughty Grin', 
'*:-)' : 'Navy Admiral', 
'@:-)' : 'Navy Captain', 
'#:-)' : 'Navy Commander', 
'=:-)' : 'Navy Lieutenant', 
'(-)' : 'Needs Haircut', 
'*8-I' : 'Nerd', 
':-B' : 'Nerd', 
'8-|' : 'Nerd', 
'8-B' : 'Nerd', 
':-=H' : 'Nerd', 
'B-|' : 'Nerd', 
'8|' : 'Nerd', 
'B|' : 'Nerd', 
'(nerd)' : 'Nerd', 
'8=|' : 'Nerd', 
'B=|' : 'Nerd', 
'|-(:-)' : 'Nerd', 
'[-_-]~' : 'Ninja', 
'Qx' : 'Ninja', 
'QK' : 'Ninja', 
'(j)' : 'Ninja', 
'(ninja)' : 'Ninja', 
'[+..]' : 'Nintendo Controller', 
'(nod)' : 'Nodding', 
'(' : 'Nodding', 
'(?_?)' : 'Nonsense', 
'):-(' : 'Nordic', 
'<:@0' : 'Nose', 
'(:^(' : 'Nose', 
'(:+)' : 'Nose', 
':/\)' : 'Nose', 
':v)' : 'Nose', 
'>*(' : 'Nose', 
'.^v' : 'Nose', 
'%-(' : 'Not Listening', 
'<2' : 'Not quite love', 
'[-(' : 'Not talking', 
'^:)^' : 'Not Worthy', 
'(8)' : 'Note', 
']:-)' : 'Nurse', 
'~=[,,_,,]:3' : 'Nyan Cat', 
'[^._.^]' : 'Nyan Cat', 
'||' : 'Nyan Cat', 
'Emoticons' : 'that start with O', 
'Emoticon' : 'Meaning', 
':' : 'Octopus', 
'O_o' : 'Oddball', 
':-j' : 'Oh Go On', 
'?:-_]' : 'Old Lady', 
'|:-{)~' : 'Old Man', 
'(-o-)' : 'Olympic Hammer Throw', 
':)]' : 'On the Phone', 
'' : 'On The Phone', 
'' : 'On The Phone', 
'()' : 'On The Phone', 
"':-(" : "One Eyebrow",
'=X' : 'Oops', 
'E:-)' : 'Operator', 
'~#|__/~~\o/~' : 'Overboard', 
'^_^' : 'Overjoyed', 
'.^_^.' : 'Overjoyed', 
':-)))' : 'Overweight', 
'^o,o^' : 'Owl', 
'-(*o*)-' : 'Owl', 
'(^o^)' : 'Owl', 
"--''--''--" : "Owl",
'-"-"-' : 'Owl', 
'^(OvO)^' : 'Owl', 
'' : 'Owl', 
'Emoticons' : 'that start with P', 
'Emoticon' : 'Meaning', 
':v' : 'Pacman', 
'(:V)' : 'Pacman', 
'(:\/)' : 'Pacman', 
'(V)=|' : 'Pacman Trophy', 
'(>_<)' : 'Painful', 
'(_)' : 'Painting', 
'8:(' : 'Panda', 
'(panda)' : 'Panda', 
'' : 'Panda', 
'\(>o<)/' : 'Panicking', 
'\\(<.<|>.>)//' : 'Panicking', 
'(>-:-)-<' : 'Parachute', 
'(@.@)' : 'Paranoid', 
'{|:{)' : 'Park Ranger', 
'<:-P' : 'Party', 
'<:o)' : 'Party', 
'<o)' : 'Party', 
'(party)' : 'Party', 
'*<(:)' : 'Party Hat', 
'<l:0' : 'Partying', 
'*<(8)~/~<' : 'Partying', 
'*<|8-P~' : 'Partying Too Much', 
'(:V' : 'Pato', 
':)>-' : 'Peace', 
'(/|\)' : 'Peace', 
'(^o^)y' : 'Peace', 
'*p*' : 'Peace', 
'(-)' : 'Peace', 
'..' : 'Peace', 
'(/)' : 'Peering', 
'<IIEEEEEED' : 'Pen', 
'0))__)>' : 'Pencil', 
'w' : 'Penguin', 
"('v')" : "Penguin",
'<(")' : 'Penguin', 
'<(^)' : 'Penguin', 
"<('')" : "Penguin",
'8>' : 'Penguin', 
'*-)' : 'Pensive', 
':-/' : 'Perplexed', 
'XP' : 'Perplexed', 
'_' : 'Perplexed', 
'<:^]' : 'Person', 
'O<-<' : 'Person', 
'>:P' : 'Phbbbbt', 
'-()' : 'Phew', 
':?)' : 'Philosopher', 
'(mp)' : 'Phone', 
'(ph)' : 'Phone', 
'(phone)' : 'Phone', 
'((()))' : 'Phone', 
'/_/_/_/_/' : 'Piano', 
'\___/' : 'Pie', 
'----|}' : 'Pie Being Thrown', 
'--===[}' : 'Pie Fight', 
'(~_(]=--Z()' : 'Pie Fight', 
'=8)' : 'Pig', 
'o()~' : 'Pig', 
':@)' : 'Pig', 
':(:)' : 'Pig', 
"<('@')>" : "Pig",
'`' : 'Pig', 
':=)' : 'Pig', 
'<(^oo^)>' : 'Pig', 
'^(*(oo)*)^' : 'Pig', 
'()' : 'Pig', 
'^..^__)~' : 'Pig', 
'(oo)' : 'Piggy', 
'/***(^o^)***\"' : 'Pigtails', 
"/~'v'~\'" : "Pigtails",
'(v)' : 'Pigtails', 
"^_^''" : "Pikachu",
'(o^-^o)' : 'Pikachu', 
'<(:-)' : 'Pilgrim', 
'(__)' : 'Pill', 
'(_)' : 'Pineapple', 
'' : 'Ping Pong table', 
':---)' : 'Pinnochio', 
':-------[' : 'Pinocchio', 
'b)' : 'Pirate', 
'P)' : 'Pirate', 
';PX' : 'Pirate', 
':ar!' : 'Pirate', 
'R)' : 'Pirate', 
'-=,q' : 'Pistol', 
'<%)' : 'Pizza', 
'(pi)' : 'Pizza', 
'<|' : 'Pizza', 
'(pizza)' : 'Pizza', 
'(pl)' : 'Plate', 
';T' : 'Playful', 
'\/' : 'Playing in the snow', 
'(+[__]:)' : 'PlayStation Portable', 
'(^^)' : 'Pleased', 
'(pu)' : 'Point Up', 
'O-G-<' : 'Pointing', 
'()' : 'Pointing', 
'' : 'Pointing', 
':-I' : 'Poker Face', 
'(*)(*O)' : 'Poking', 
'' : 'Polar bear', 
'-[[]]--[()]-' : 'Police Chase', 
'[|:-{O' : 'Police Officer', 
'c|:-|' : 'Police Officer', 
'()' : 'Pompous', 
'^' : '', 
'' : 'Poodle', 
'(poolparty)' : 'Pool Party', 
'(hrv)' : 'Pool Party', 
'~@~' : 'Poop', 
'+O=-)' : 'Pope', 
'+<:-)' : 'Pope', 
';-?' : 'Popeye', 
':<' : 'Pout', 
'>:-[' : 'Pouting', 
':-\"' : 'Pouting', 
'\o/' : 'Praise The Lord', 
'O/' : 'Praise The Lord', 
'_/\_' : 'Prayer', 
"/\'s" : "Prayers",
'[-o<' : 'Praying', 
'_/\_' : 'Praying', 
'_|' : 'Praying', 
'(*)' : 'Praying', 
'' : 'Precious', 
'&' : 'Pretzel', 
'+:-)' : 'Priest', 
'(zilmer)' : 'Priidu', 
'\<><>/' : 'Prince', 
'O(+>' : 'Prince', 
'~<:-)' : 'Princess', 
'\&&&/' : 'Princess', 
':}' : 'Princess', 
'@(*-*)@' : 'Princess Leia', 
'x{|:)' : 'Propeller Hat', 
'-' : 'Proud', 
'(+[PSP]::)' : 'PSP', 
'' : 'Psycho', 
'*3' : 'Pucker up', 
':&' : 'Puke', 
'(puking)' : 'Puking', 
'(^:^)' : 'Pumpkin', 
'(~~)' : 'Pumpkin', 
'*|' : 'Punch', 
'(_)' : 'Punched', 
'(punch)' : 'Punching', 
'(o(o' : 'Punching', 
'=(-_-' : 'Punching', 
'0()=' : 'Punching', 
'(o)=)`)' : 'Punching', 
'()=)-)' : 'Punching', 
'-:-/' : 'Punk', 
'UU' : 'Puppy', 
'__' : 'Puppy Love', 
"Q('.'Q)" : "Put Em Up",
'()' : 'Puzzled', 
'Emoticons' : 'that start with Q', 
'Emoticon' : 'Meaning', 
'C):-' : 'O', 
'C):-' : 'O', 
'C):-' : 'O', 
'C):-O' : 'Quartet', 
'\%%%/' : 'Queen', 
':?:' : 'Question', 
'Emoticons' : 'that start with R', 
'Emoticon' : 'Meaning', 
'>-=]' : 'Rabbit', 
"('.')" : "Rabbit",
'(")_(")' : 'Rabbit', 
'(uu)' : 'Rabbit', 
'8:3' : 'Rabbit', 
'`=o=^o>' : 'Race Car', 
'CD:-)>' : 'Railroad Worker', 
'~~:-(' : 'Rain', 
'(rain)' : 'Rain', 
'(london)' : 'Rain', 
'(R)' : 'Rainbow', 
'/:)' : 'Raised Eyebrow', 
'd:o)' : 'Rapper', 
'=:7)~~' : 'Rat', 
'=:7(~~' : 'Rat', 
'(((((:-{=' : 'Raver', 
'_' : 'Raving', 
'(l)' : 'Red Heart', 
'(k)' : 'Red Lips', 
'(f)' : 'Red Rose', 
'}---:o>' : 'Reindeer', 
'o(^)o' : 'Reluctant', 
'~o)' : 'Reproduction', 
'>=D' : 'Revenge', 
'?$?:-)' : 'Rich', 
'(})' : 'Right Hug', 
'/O' : 'Rimshot', 
'(_)' : 'Ringo Starr', 
'________' : 'Road', 
'*<l:o)>' : 'Robin Hood', 
'([(' : 'Robocop', 
'[*.*]' : 'Robot', 
':|]' : 'Robot', 
':]' : 'Robot', 
':]]' : 'Robot', 
'[:]' : 'Robot', 
'<(-_-)>' : 'Robot', 
'~(o_o)~' : 'Robot', 
'[:|]' : 'Robot', 
'\m/(**)\m/' : 'Rock On', 
'\m/@_@\m/' : 'Rock On', 
'\m/>.<\m/' : 'Rocker', 
'\m/(><)\m/' : 'Rocker', 
'd(>_<)b' : 'Rocker', 
'\m/(>.<)\m/' : 'Rocker', 
'bd(OoO)bd' : 'Rocker', 
'<:-)<<|' : 'Rocket', 
'<===~~' : 'Rocket', 
'\w/^^,\w/' : 'Rocking', 
'>-D' : 'Rocking', 
'\m/' : 'Rocking', 
'\\m/(=o+)\m//' : 'Rocking', 
'(rock)' : 'Rocking', 
'\m/(*o*)\m/' : 'Rocking Out', 
'\m/d(^_^)b\m/' : 'Rocking Out', 
'8-|' : 'Rolling Eyes', 
'>>' : 'Rolling Eyes', 
'9_9' : 'Rolling Eyes', 
'@@' : 'Rolling Eyes', 
'c.c' : 'Rolling Eyes', 
':roll:' : 'Rolling Eyes', 
'' : 'Rolling Eyes', 
'=))' : 'Rolling on the Floor Laughing', 
'(rotfl)' : 'Rolling on the Floor Laughing', 
'(rofl)' : 'Rolling on the Floor Laughing', 
'()' : 'Romantic', 
'7:-)' : 'Ronald Reagan', 
'\:)' : 'Ronald Reagan', 
'~:>' : 'Rooster', 
'^:3' : 'Rooster', 
'@)->-' : 'Rose', 
'@~)~~~~' : 'Rose', 
'@}->--' : 'Rose', 
'@};-' : 'Rose', 
"@~'~~~" : "Rose",
'@>-;--' : 'Rose', 
'@-->---' : 'Rose', 
'--{--@' : 'Rose', 
'@>--' : 'Rose', 
'@}-^--^--' : 'Rose', 
'(@)->->->--' : 'Rose', 
'@}-(' : 'Rose', 
'<(*_*)>' : 'Rosie', 
'3:*>' : 'Rudolf', 
'(;_)' : 'Running', 
'=()' : 'Running', 
'' : 'Running', 
'=(;)' : 'Running', 
'Emoticons' : 'that start with S', 
'Emoticon' : 'Meaning', 
'=(' : 'Sad', 
':-(' : 'Sad', 
':(' : 'Sad', 
'T.T' : 'Sad', 
'=[' : 'Sad', 
':c' : 'Sad', 
'D:' : 'Sad', 
'D=' : 'Sad', 
'Y_Y' : 'Sad', 
'-' : 'Sad', 
'v.v' : 'Sad', 
'\8^1' : 'Sad', 
'(<_>)' : 'Sad', 
'(/_\)' : 'Sad', 
'):' : 'Sad', 
'(_)' : 'Sad', 
':[' : 'Sad', 
':sad:' : 'Sad', 
':=(' : 'Sad', 
'(sad)' : 'Sad', 
'8-(' : 'Sad', 
'/:-)' : 'Sailor', 
';-)}<////>' : 'Salesman', 
'>>>>(:-)' : 'Salesman', 
'M:-)' : 'Salute', 
'O7' : 'Salute', 
':-{o' : 'Salvador Dali', 
'*<]:{)' : 'Santa', 
'*<(:-{)}' : 'Santa', 
'*<:-)' : 'Santa Claus', 
'*<|:-{)}' : 'Santa Claus', 
'<|:)}' : 'Santa Claus', 
'0<[]:-)>>' : 'Santa Claus', 
'*-={:-)}}}>' : 'Santa Claus', 
'*<|:o)>' : 'Santa Claus', 
'<[]:o{|>' : 'Santa Claus', 
'*<[]:o{|>' : 'Santa Claus', 
':-7' : 'Sarcastic', 
'^o)' : 'Sarcastic', 
';s' : 'Sarcastic', 
':S' : 'Sarcastic', 
'D:' : 'Scared', 
':!' : 'Scared', 
'(;)' : 'Scared', 
'.()' : 'Scared', 
'()' : 'Scaring', 
'()' : 'Scary', 
'' : 'Scheming', 
'*!#*!^*&:-)' : 'Schizophrenic', 
'!:-)' : 'Scholar', 
'*s*' : 'School', 
'8<' : 'Scissors', 
'c--I-I-I{' : 'Scorpion', 
'3#`' : 'Scorpion', 
'>-$' : 'Scowling', 
'^:-/' : 'Scratching Head', 
':-@' : 'Screaming', 
'..._[:]' : 'Scuba Diver', 
'..._(:)-o' : 'Scuba Diver', 
'O-)' : 'Scuba Diver', 
'(:)-)' : 'Scuba Diver', 
"\';./" : "Sea Horse",
'' : 'Seal', 
'()' : 'Seal', 
';^!' : 'Secret', 
':-*' : 'Secret Telling', 
'8==8' : 'Seeing Eye to Eye', 
'\000/' : 'Shaka', 
'(shake)' : 'Shaking', 
'9(x.x)9' : 'Shaking Fist', 
'(x.x)9' : 'Shaking Fist', 
'[-x' : 'Shame on You', 
'<(((">' : 'Shark', 
'(^^^)' : 'Shark', 
'~~~/\\~~~' : 'Shark', 
'>---^-*<' : 'Shark', 
'_/\____\0/__' : 'Shark', 
'_/\_\o/_' : 'Shark Attack', 
'=O' : 'Shocked', 
'*SHOCKED*' : 'Shocked', 
'o_o' : 'Shocked', 
':-o' : 'Shocked', 
'8-0' : 'Shocked', 
'o' : 'Shocked', 
'O' : 'Shocked', 
'8^{}' : 'Shocked', 
'(O_O)' : 'Shocked', 
'8O' : 'Shocked', 
':shock:' : 'Shocked', 
'(**)' : 'Shocked', 
'' : 'Shocked', 
'=(.)=r' : 'Shooting', 
'()o*--' : 'Shooting', 
'()*' : 'Shooting', 
'' : 'Shooting Star', 
'^^' : 'Shoulder Shrug', 
':-V' : 'Shouting', 
':\/' : 'Shouting', 
'|----D' : 'Shovel', 
"`\('_')/`" : "Shrug",
'\(_)/' : 'Shrugging', 
'\_()_/' : 'Shrugging', 
'(*^_^*)' : 'Shy', 
'-_-' : 'Shy', 
'(^^;)' : 'Shy', 
'(#^.^#)' : 'Shy', 
'(#/\#)' : 'Shy', 
'(*)' : 'Shy', 
'(-:|:-)' : 'Siamese Twins', 
':-&' : 'Sick', 
'+o(' : 'Sick', 
'|-~b' : 'Sick', 
'(-__-)' : 'Sick', 
':-~)' : 'Sick', 
':->|' : 'Sick', 
':-Y' : 'Side Comment', 
':-<' : 'Sigh', 
'8-}' : 'Silly', 
'=P' : 'Silly', 
'C(:' : 'Silly', 
'(^o^)' : 'Singing', 
':-o' : 'Singing', 
'(o)~' : 'Singing', 
']:' : 'Skateboard', 
'O-\-<]:' : 'Skateboarder', 
'0-<-<):' : 'Skateboarder', 
'o{-<]:' : 'Skateboarder', 
'O[-<]:' : 'Skateboarder', 
'o<[^(' : 'Skater', 
'o->-</:' : 'Skater', 
'q=)-<--<|:' : 'Skater', 
'O-/-<]:' : 'Skater', 
':-/' : 'Skeptical', 
':-7' : 'Skeptical', 
':/' : 'Skeptical', 
'(.)' : 'Skeptical', 
'*(H-)' : 'Skier', 
'=(^-^)' : 'Skipping', 
'8-X' : 'Skull', 
'(skype)' : 'Skype', 
'(ss)' : 'Skype', 
'(^^)(o)' : 'Slapping', 
'(>.<)' : 'zzZ Sleeping', 
'-_-' : 'Sleeping', 
'-_-' : 'Zzzzz Sleeping', 
'(-.-)zzZZ' : 'Sleeping', 
'Z/' : 'Sleeping', 
'Z_Z' : 'Sleeping', 
'Q' : 'Q', 
'-' : 'Sleeping', 
'(~_~)^' : 'Sleeping', 
'(-_o)zzZ' : 'Sleeping', 
'O-(==<' : 'Sleeping', 
'<-.->' : 'Sleeping', 
'(--)zzz...' : 'Sleeping', 
'' : 'Sleeping', 
'|-)' : 'Sleepy', 
'(>.<)' : 'Sleepy', 
'u_u' : 'Sleepy', 
'#)' : 'Sleepy', 
'"8^y' : 'Sleepy', 
'~_~' : 'Sleepy', 
'(-_-)Zzz' : 'Sleepy', 
'(Z.Z)' : 'Sleepy', 
'I-)' : 'Sleepy', 
'(snooze)' : 'Sleepy', 
'I=)' : 'Sleepy', 
"#(,'%/)" : "Slept Too Long",
':-#' : 'Smacked', 
':-)' : 'Smile', 
':)' : 'Smile', 
'o,' : 'o', 
'\_/' : 'Smile', 
'[______]' : 'Smile', 
':|)' : 'Smile', 
'C:' : 'Smile', 
'\\)' : 'Smile', 
':A' : 'Smile', 
':)' : 'Smile', 
'=]' : 'Smile', 
'\__/' : 'Smile', 
'(*8' : 'Smile', 
'I:(' : 'Smile', 
'>-)' : 'Smile', 
':^D' : 'Smile', 
':]' : 'Smile', 
':smile:' : 'Smile', 
':=)' : 'Smile', 
'(smile)' : 'Smile', 
'()' : 'Smile', 
':^J' : 'Smiling', 
'^^' : 'Smiling', 
'^&^' : 'Smiling', 
".'J" : "Smiling",
':-1' : 'Smirk', 
':-,' : 'Smirk', 
':^,' : 'Smirking', 
';^)' : 'Smirking', 
'(smirk)' : 'Smirking', 
'()' : 'Smitten', 
':-p~' : 'Smoker', 
':-d~' : 'Smoker', 
':-Q' : 'Smoking', 
':O\"' : 'Smoking', 
':-' : 'i Smoking', 
':-?' : 'Smoking', 
':-d' : 'Smoking', 
':-G-' : 'Smoking', 
'(--)y-' : 'Smoking', 
'(())y-' : 'Smoking', 
'(smoke)' : 'Smoking', 
'(smoking)' : 'Smoking', 
'(ci)' : 'Smoking', 
'o()y' : 'Smoking', 
':-?' : 'Smoking a Pipe', 
'(`)' : 'Smooch', 
':>' : 'Smug', 
'B|' : 'Smug', 
'(sn)' : 'Snail', 
'@x' : 'Snail', 
'_@/' : 'Snail', 
'`@____' : 'Snail', 
'@_v' : 'Snail', 
'@_' : 'Snail', 
'_@_v' : 'Snail', 
'"' : 'Snail', 
'_@v' : 'Snail', 
'~<:<<<<<>' : 'Snake', 
'~~~~8}' : 'Snake', 
'~>)' : 'Snake', 
':-`|' : 'Sniffling', 
':<)' : 'Snobby', 
'(_' : 'Snooper', 
'|^O' : 'Snoring', 
'^o' : 'Snoring', 
'' : 'Snowing', 
'(:^)_)_)' : 'Snowman', 
'[l:^)' : 'Snowman', 
'(:-)()' : 'Snowman', 
'*|:^)(.)(...)' : 'Snowman', 
'(`)' : 'Snubbing', 
'(A`)' : 'Snubbing', 
'' : 'O', 
'D' : '', 
'' : 'O So Wrong', 
'.`(><)`.' : 'Sobbing', 
'(so)' : 'Soccer Ball', 
'(____)' : 'Sock', 
'/:-)' : 'Soldier', 
'c]:{D' : 'Sombrero', 
'' : 'Sonic the Hedgehog', 
'?-(' : 'Sorry', 
'=^{' : 'Sorry', 
'>.<' : 'Sour', 
':-*' : 'Sour', 
':-6' : 'Sour', 
'(X^(*' : 'Sour Face', 
'X' : 'P Sour Face', 
'=.=' : 'Sour Face', 
'[>==]' : 'South African Flag', 
'0_o' : 'Spaced Out', 
'|-=[O]=-|' : 'Spaceship', 
':-M' : 'Speak no Evil', 
'=#' : 'Speechless', 
':|' : 'Speechless', 
':-|' : 'Speechless', 
':=|' : 'Speechless', 
'(speechless)' : 'Speechless', 
'o' : 'O', 
'||||' : 'Spider', 
'/\(00)/\"' : 'Spider', 
'' : 'Spider', 
'^;;^' : 'Spider', 
'::::(' : 'Spider', 
':-`' : 'Spitting', 
'>:-I' : 'Spock', 
'_' : 'Spooked', 
'===()' : 'Sprinting', 
'__m_OO_m__' : 'Spy', 
'<:' : 'Squid', 
'' : 'Squid', 
'(><)' : 'Squinting', 
'-_-' : 'Squinting', 
'(*)' : 'Star', 
'(star)' : 'Star', 
'*~*' : 'Star Gazed', 
'' : 'Staring', 
'(;_)' : 'Staring', 
'()' : 'Staring', 
'%.)' : 'Staring at a Screen for Too Long', 
'=-O' : 'Starship Enterprise', 
'(~);}' : 'Stealie', 
';@' : 'Steaming Mad', 
'||=||' : 'Stereo', 
'|:-|' : 'Stern', 
'' : 'O', 
'/\"' : 'Stick Man', 
'' : 'O', 
'#:-)-{--<' : 'Stick Man', 
'>-/-@' : 'Stick Man', 
'O-[--<' : 'Stickfigure', 
':-)~' : 'Sticking Tongue Out', 
':-P' : 'Sticking Tongue Out', 
':P' : 'Sticking Tongue Out', 
'>:P' : 'Sticking Tongue Out', 
':-r' : 'Sticking Tongue Out', 
':pp' : 'Sticking Tongue Out', 
':(,)' : 'Sticking Tongue Out', 
'(:-p)' : 'Sticking Tongue Out', 
'' : 'Sticking Tongue Out', 
".'r" : "Sticking Tongue Out",
':razz:' : 'Sticking Tongue Out', 
':=P' : 'Sticking Tongue Out', 
'(tongueout)' : 'Sticking Tongue Out', 
'~/~' : 'Stirring', 
'$:-$' : 'Stock Broker', 
'>;D' : 'Stoked', 
'*STOP*' : 'Stop', 
'(st)' : 'Stormy Cloud', 
':|' : 'Straight face', 
".'T" : "Straight Face",
'-.-' : 'Stressed', 
'O=[O{|___|]=O' : 'Stretcher', 
'@?@' : 'Stunned', 
'[o].[o]' : 'Stunned', 
'&;-P' : 'Suave', 
'(sun)' : 'Sun', 
'8-)' : 'Sunglasses', 
'B-)' : 'Sunglasses', 
'B-D' : 'Sunglasses', 
']-I' : 'Sunglasses', 
'B:-)' : 'Sunglasses', 
'8-|' : 'Sunglasses', 
'8|' : 'Sunglasses', 
'B-|' : 'Sunglasses', 
'B|' : 'Sunglasses', 
'8)' : 'Sunglasses', 
':cool:' : 'Sunglasses', 
'c-oo' : 'Sunglasses', 
'_\|/_' : 'Sunset', 
'(#)' : 'Sunshine', 
'((o))' : 'Sunshine', 
':o' : 'Surprised', 
':-O' : 'Surprised', 
'*SURPRISED*' : 'Surprised', 
':O' : 'Surprised', 
'{:o' : 'Surprised', 
'(:)o)' : 'Surprised', 
'O-' : 'O', 
'U' : 'Surprised', 
':-()' : 'Surprised', 
'=0' : 'Surprised', 
'G' : 'G', 
'-' : 'Surprised', 
'-' : 'Surprised', 
'(*.*)' : 'Surprised', 
'l8^O' : 'Surprised', 
'****' : 'Surprised', 
'0' : 'Surprised', 
'' : 'Surprised', 
'*_*' : 'Surprised', 
':eek:' : 'Surprised', 
':=o' : 'Surprised', 
'(surprised)' : 'Surprised', 
'*.*' : 'Surprised', 
'(O.O)' : 'Surprised', 
'=^o' : 'Surprised', 
'=^0' : 'Surprised', 
'' : 'Surrender', 
'' : 'O Suspicious', 
'' : 'Suspicious', 
':#' : 'Swearing', 
'>:-@!' : 'Swearing', 
'(swear)' : 'Swearing', 
'`:-)' : 'Sweating', 
'(:|' : 'Sweating', 
'(sweat)' : 'Sweating', 
'8-)' : 'Swimmer', 
'__/\o_' : 'Swimming', 
'B-)-[<' : 'Swimming Trunks', 
'x_O' : 'Swollen Eye', 
'/________|___|' : 'Sword', 
'oxx:{}:::::::>' : 'Sword', 
'cxxx{}:::::::>' : 'Sword', 
'0]xxxx]====>' : 'Sword', 
'<=======|==o' : 'Sword', 
'(>_)(_<)' : 'Sword Fighting', 
'---(|||]==[]' : 'Syringe', 
'+=>-' : 'Syringe', 
'Emoticons' : 'that start with T', 
'Emoticon' : 'Meaning', 
'/(..)' : 'Table Flips You', 
'(>o<)' : 'Table Tennis', 
'=;' : 'Talk to the Hand', 
':^y' : 'Talking', 
':y' : 'Talking', 
'(talk)' : 'Talking', 
'(____)' : 'Tank', 
'|"""\-=' : '', 
'(____)' : 'Tank', 
'>8-O-(&)' : 'Tapeworm', 
'()' : 'Tasty', 
'(tuari)' : 'Tauri', 
'\_/?' : 'Tea Cup', 
'C(_)~' : 'Teapot', 
"C(_)~'" : "Teapot",
':p' : 'Teasing', 
'B^P' : 'Teasing', 
':^P' : 'Teasing', 
'8^P' : 'Teasing', 
':O)' : 'Teddy Bear', 
'()' : 'Teddy Bear', 
'(t)' : 'Telephone Receiver', 
'(((((^_^)' : 'Teleporting', 
'-(:)(0)=8' : 'Teletubby', 
'>[I' : 'Television', 
'ixxxxxi' : 'Tennis Net', 
'|B-)' : 'Terminator', 
'-:}' : 'The Grinch', 
'8=:-K' : 'The Swedish Chef', 
':-?' : 'Thinking', 
'*-)' : 'Thinking', 
':?' : 'Thinking', 
':=?' : 'Thinking', 
'(think)' : 'Thinking', 
'(*_)*' : 'Throwing Away', 
'(n)' : 'Thumbs Down', 
"q('-')p" : "Thumbs Down",
':-q' : 'Thumbs Down', 
'(no)' : 'Thumbs Down', 
'(y)' : 'Thumbs Up', 
"d'-'" : "Thumbs Up",
"d'-'b" : "Thumbs Up",
':-bd' : 'Thumbs Up', 
'(^-^)b' : 'Thumbs Up', 
'd-(^.^)z' : 'Thumbs Up', 
'(ok)' : 'Thumbs Up', 
'(yes)' : 'Thumbs Up', 
'_' : 'Thumbs Up', 
':-)<////>' : 'Tie', 
'|-(o)-|' : 'TIE Fighter', 
'(-o-)' : 'TIE Fighter', 
'|o|' : 'TIE Fighter', 
'o' : '', 
'o' : 'Tigger', 
':-T' : 'Tight Lipped', 
'[]---|<O---[]' : 'Tight Rope', 
'(o)' : 'Time', 
'(time)' : 'Time', 
'(clock)' : 'Time', 
':-t' : 'Time Out', 
'-<:-(' : 'Tin Man', 
'(:|' : 'Tired', 
'*TIRED*' : 'Tired', 
'=_=' : 'Tired', 
'%-|' : 'Tired', 
'%-/' : 'Tired', 
'(z_z~.)' : 'Tired', 
'()' : 'Tired', 
'_' : 'Tired', 
'(=_=)' : 'Tired', 
'(%]:D' : 'Toad', 
'0000(0)(0)0000' : 'Toes', 
'.oooO' : 'Toes', 
'Oooo.' : 'Toes', 
'(toivo)' : 'Toivo', 
".'w" : "Tongue",
'8-;)' : 'Tongue in Cheek', 
':-J' : 'Tongue in Cheek', 
':j' : 'Tongue in Cheek', 
':-&' : 'Tongue Tied', 
':-a' : 'Tongue Touching Nose', 
'(tmi)' : 'Too Much Information', 
'[|:)' : 'Top Hat', 
'#:o\:o/:o\:o/:o||' : 'Totem Pole', 
"=')" : "Touched",
'{(:-)' : 'Toupee', 
'}:-(' : 'Toupee', 
'{:-)' : 'Toupee', 
'-HHHH' : 'Tower', 
'[..]' : 'Transformer', 
'>~<' : 'Translate', 
'>:-|' : 'Trekkie', 
':<()' : 'Tribesman', 
'---(o__)o' : 'Tripping', 
'(>_<)>' : 'Troubled', 
'(^_^;)' : 'Troubled', 
'<.<' : 'Troublemaker', 
'__' : '', 
'' : 'O    O Truck', 
'oo---oo-Bo' : 'Truck', 
'-=iii=<()' : 'Trumpet', 
'(_)' : 'Tuba', 
'O>-O>-|-<O-<O' : 'Tug of War', 
'X:-|' : 'Tupac Shakur', 
'@=)' : 'Turban', 
'@:-v' : 'Turban', 
'<:>==' : 'Turkey', 
'mm' : '', 
'' : 'l   L Turkey', 
'=8>-()<' : 'Turkey', 
'(tu)' : 'Turtle', 
':(' : 'Turtle', 
'/Y\"' : 'Tuxedo', 
':-)8---' : 'Tuxedo', 
':-)8>' : 'Tuxedo', 
'Emoticons' : 'that start with U', 
'Emoticon' : 'Meaning', 
'.-=-.' : 'UFO', 
'(o|o)' : 'Ultraman', 
'(um)' : 'Umbrella', 
'(---,' : 'Umbrella', 
'q;|' : 'Umpire', 
'x:-/' : 'Uncertain', 
'=l:-)=' : 'Uncle Sam', 
'=):-)>' : 'Uncle Sam', 
':-\"' : 'Undecided', 
':\"' : 'Undecided', 
'/:-|' : 'Undecided', 
':<' : 'Unhappy', 
'|:-)' : 'Unibrow', 
'|:' : 'Unimpressed', 
'' : 'Unimpressed', 
'**==' : 'United States Flag', 
">->'" : "Unsure",
':/' : 'Unsure', 
'8-\"' : 'Unsure', 
'>.<' : 'Upset', 
'}:(' : 'Upset', 
'>:*' : 'Upset', 
'D:<' : 'Upset', 
'>=P' : 'Upset', 
'>-:O' : 'Upset', 
'--"' : 'Upset', 
'.-.' : 'Upside Down', 
'P:' : 'Upward Lick', 
'Emoticons' : 'that start with V', 
'Emoticon' : 'Meaning', 
':-E' : 'Vampire', 
"(';.;')" : "Vampire",
'@,.,@' : 'Vampire', 
'*,..,*' : 'Vampire', 
'-,..,-' : 'Vampire', 
':-{' : 'Vampire', 
':f' : 'Vampire', 
"',..,'" : "Vampire",
'(o,..,o)' : 'Vampire', 
'(=,..,=)' : 'Vampire', 
':-[' : 'Vampire', 
':(=' : 'Vampire', 
'8-E' : 'Vampire', 
'>:-E' : 'Vampire', 
'>8-E' : 'Vampire', 
'>:-[' : 'Vampire', 
'8-{' : 'Vampire', 
'>8-{' : 'Vampire', 
':-[' : 'Vampire Bat', 
':[' : 'Vampire Bat', 
'^,..,^' : 'Vampire Bat', 
'^,..,^' : 'Vampire Bat', 
'~;;~' : 'Vampire Bat', 
'(^-^:)' : 'Vampire Bite', 
':-[' : 'Vampiro', 
'%*}' : 'Very Drunk', 
'=D' : 'Very Happy', 
':-))' : 'Very Happy', 
':-((' : 'Very Sad', 
'(o^_^)o' : 'Victory', 
'v(^_^)v' : 'Victory', 
'()' : 'Victory', 
'8<' : 'Video Projector', 
'](-[3' : 'Viking', 
'(o\_!_/o)' : 'Volkswagen Bug', 
'(o^)/(^o)' : 'Volleyball', 
'' : 'o', 
'ixxxxxxx' : 'i', 
':O=' : 'Vomit', 
':-C..' : 'Vomiting', 
'Emoticons' : 'that start with W', 
'Emoticon' : 'Meaning', 
"(>'.')>#" : "Waffle",
"(>'_')>#" : "Waffle",
'#<(0.0<)' : 'Waffle', 
'(o*)' : 'Wait', 
'(wait)' : 'Wait', 
'(((((((`)' : 'Wait', 
'(_)' : 'Wait', 
':->X==|' : 'Waiter', 
'_()' : 'Waiter', 
':-w' : 'Waiting', 
':3=' : 'Walrus', 
'(:{C)' : 'Walrus', 
':-<' : 'Walrus', 
'(:3=' : 'Walrus', 
'._)' : 'Warping', 
'-' : 'Watching', 
'__/\o_' : 'o Water Polo', 
':-h' : 'Wave', 
'(wave)' : 'Wave', 
'(hi)' : 'Wave', 
'(bye)' : 'Wave', 
'~~c___' : 'Waves', 
':-)--' : 'Weakling', 
'(~`)' : 'Weary', 
';w;' : 'Weeping', 
'(;`)' : 'Weeping', 
'(><)' : 'Weeping', 
':-P' : 'Whatever', 
'-_-' : 'Whatever', 
':-1' : 'Whatever', 
'(_)' : 'Whatever', 
'(_)' : 'Whatever', 
'.' : 'Whatever', 
'd-:' : 'Whatever', 
'#:-S' : 'Whew', 
'(whew)' : 'Whew', 
":'(" : "Whining",
":'-(" : "Whining",
'*WHINING*' : 'Whining', 
'(`)' : 'Whispering', 
':-"' : 'Whistling', 
".'Y" : "Whistling",
'()' : 'Whistling', 
'()' : 'Whoops', 
'(w)' : 'Wilted Rose', 
'' : 'Windows', 
'())=(' : 'Wine Glass', 
';p' : 'Wink', 
';-P' : 'Wink', 
';^)' : 'Wink', 
'(`_^)' : 'Wink', 
',-)' : 'Wink', 
'(^_~)' : 'Wink', 
'9-)' : 'Wink', 
'6-)' : 'Wink', 
'i)' : 'Wink', 
'!)' : 'Wink', 
';-)' : 'Winking', 
';)' : 'Winking', 
'~_^' : 'Winking', 
'U' : 'Winking', 
';()' : 'Winking', 
':wink:' : 'Winking', 
';=)' : 'Winking', 
'(wink)' : 'Winking', 
'' : 'Winking', 
'(^)' : 'Winking', 
'!-)' : 'Winking', 
'(-!' : 'Winking', 
'^_^a' : 'Wiping Tears', 
'' : 'Wiping Tears', 
'<):^/' : 'Witch', 
'___' : 'Witch', 
'<|:~,' : 'Witch', 
'I8${}' : 'Witch Doctor', 
'8<:-)' : 'Wizard', 
'<{{{};->~' : 'Wizard', 
'<*(:-?' : 'Wizard', 
'(-).*' : 'Wizard', 
'{0-|<[' : 'Woman', 
'o>-<|=' : 'Woman', 
'o.o?' : 'Wondering', 
':^)' : 'Wondering', 
'(wonder)' : 'Wondering', 
'||' : 'Woody Allen', 
'(wo)' : 'Work', 
':-S' : 'Worried', 
'~' : 'Worried', 
'(;)' : 'Worried', 
'>w<' : 'Worried', 
':S' : 'Worried', 
':=S' : 'Worried', 
'(worry)' : 'Worried', 
'' : 'Worried', 
'(`)' : 'Worried', 
'()' : 'Worried', 
'()' : 'Worshiping', 
'(_;)' : 'Writing', 
'()' : 'Writing', 
'Emoticons' : 'that start with X', 
'Emoticon' : 'Meaning', 
'(xx)' : 'Xbox', 
'[x]' : 'Xbox', 
'Emoticons' : 'that start with Y', 
'Emoticon' : 'Meaning', 
'_/)' : 'Yacht', 
'|-O' : 'Yawn', 
'(:|' : 'Yawn', 
".'U" : "Yawn",
'(yawn)' : 'Yawning', 
'(;`)' : 'Yawning', 
'*\o/*' : 'Yay', 
'>:O' : 'Yelling', 
'=^Y' : 'Yelling', 
':-(0)' : 'Yelling', 
':(*)' : 'Yelling', 
'(*O)' : 'Yelling', 
'(__)' : 'Yikes', 
'(%)' : 'Yin Yang', 
'\(o/' : 'Yin Yang', 
'<(-_-)>' : 'Yoda', 
'<(-.-)>' : 'Yoda', 
':-0>' : 'Yuck Face', 
'()' : 'Yummy', 
'Emoticons' : 'that start with Z', 
'Emoticon' : 'Meaning', 
':#' : 'Zip It', 
'F(x_x)F' : 'Zombie', 
'8-#' : 'Zombie', 
'[-]' : 'Zombie', 
'Z' : 'Zorro', 
'B-)===>' : 'ZZ Top', 
}